<a href="https://colab.research.google.com/github/Adlucent/ga4-return-prediction/blob/main/3_Order_Level_Agg_and_Feature_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3. Order-Level Aggregation & Feature Engineering
<br>
1. Define GCP Variables<br>
2. Import Libraries and Load Dataset saved in Notebook 2<br>
3. Aggregate Transaction-Level and Item-Level Features<br>
4. Aggregate Session-Level Features<br>
5. Join Transaction-Level and Session-Level Features Back Together<br>
6. Split Pre-Transaction Session and Transaction Session rows, and then aggregate and prepare each df<br>
7. Merge Tables Back Together and Save to BQ<br>

## Define GCP Variables

In [ ]:
################################################################################
######################### CHANGE BQ PROJECT NAME BELOW #########################
################################################################################

project_name = 'adl-analytics' #add proj name
region = 'US'  # GCP project region

## Import Libraries

In [ ]:
# Google credentials
from google.colab import auth
auth.authenticate_user()

# BigQuery Magics
'''BigQuery magics are used to run BigQuery SQL queries in a python environment.
These queries can also be run in the BigQuery UI '''

from google.cloud import bigquery
from google.cloud.bigquery import magics, Client, QueryJobConfig

magics.context.project = project_name
client = bigquery.Client(project=magics.context.project)

import pandas as pd
import numpy as np
from datetime import timedelta
from functools import reduce

import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

# suppress notebook warnings
import warnings
warnings.filterwarnings('ignore')

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.float_format", lambda x: "%.3f" % x)

## Load BigQuery Table Saved in Last Notebook

In [ ]:
%%bigquery data --project $project_name
SELECT *
FROM `adl-analytics.return_prediction_ga4.return_prediction_ga4_1c` # update with your project, dataset name & table name

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
# create a copy to avoid re-importing in case you need to revert back
df = data.copy()

In [ ]:
# check row and column count
df.shape

(2002010, 110)

In [ ]:
# check first five rows
df.head()

,event_date,event_params_engagement_time_msec,event_params_ga_session_id,event_params_ga_session_number,event_params_session_engaged,user_pseudo_id,user_ltv_revenue,ecommerce_total_item_quantity,total_return_item_quantity,ecommerce_purchase_revenue_in_usd,ecommerce_tax_value_in_usd,ecommerce_unique_items,ecommerce_transaction_id,item_price_in_usd,item_promotion_name,item_quantity,item_refund_in_usd,item_refund_quantity,item_revenue_in_usd,event_name_add_to_cart,event_name_begin_checkout,event_name_page_view,event_name_scroll,event_name_select_item,event_name_user_engagement,event_name_view_item,device_category_desktop,device_category_mobile,device_mobile_brand_name_Apple,device_mobile_brand_name_Google,device_mobile_brand_name_Huawei,device_mobile_brand_name_Microsoft,device_mobile_brand_name_Mozilla,device_mobile_brand_name_Samsung,device_mobile_brand_name_Xiaomi,device_mobile_model_name_Chrome,device_mobile_model_name_ChromeBook,device_mobile_model_name_Edge,device_mobile_model_name_Firefox,device_mobile_model_name_Safari,device_mobile_model_name_iPad,device_mobile_model_name_iPhone,device_web_info_browser_AndroidWebview,device_web_info_browser_Chrome,device_web_info_browser_Edge,device_web_info_browser_Firefox,device_web_info_browser_Safari,geo_country_Canada,geo_country_France,geo_country_India,geo_country_Other,geo_country_Spain,geo_country_UnitedKingdom,geo_country_UnitedStates,traffic_source_medium_Other,traffic_source_medium_cpc,traffic_source_medium_organic,traffic_source_medium_referral,event_params_parent_page_Apparel,event_params_parent_page_CampusCollection,event_params_parent_page_CheckoutConfirmation,event_params_parent_page_CheckoutYourInformation,event_params_parent_page_EcoFriendly,event_params_parent_page_Home,event_params_parent_page_Lifestyle,event_params_parent_page_New,event_params_parent_page_Other,event_params_parent_page_PaymentMethod,event_params_parent_page_Sale,event_params_parent_page_ShopbyBrand,event_params_parent_page_ShoppingCart,event_params_parent_page_Stationery,event_params_child_page_Bags,event_params_child_page_Drinkware,event_params_child_page_Google,event_params_child_page_Hats,event_params_child_page_Kids,event_params_child_page_MensUnisex,event_params_child_page_Notebooks,event_params_child_page_Other,event_params_child_page_SmallGoods,event_params_child_page_Socks,event_params_child_page_Stickers,event_params_child_page_Womens,event_params_child_page_Writing,event_params_child_page_YouTube,item_parent_category_Apparel,item_parent_category_Collections,item_parent_category_Lifestyle,item_parent_category_New,item_parent_category_Other,item_parent_category_Sale,item_parent_category_ShopbyBrand,item_parent_category_Stationery,item_child_category_Bags,item_child_category_CampusCollection,item_child_category_Drinkware,item_child_category_Google,item_child_category_Kids,item_child_category_MensUnisex,item_child_category_Other,item_child_category_SmallGoods,item_child_category_Womens,item_child_subcategory_Backpacks,item_child_subcategory_ElectronicsAccessories,item_child_subcategory_Infant,item_child_subcategory_MensTShirts,item_child_subcategory_MugsTumblers,item_child_subcategory_Other,item_child_subcategory_WaterBottles
0,2020-12-05,2275,1503908547,1,1,77375656.7088433360,0.000,1,0,0.000,0.000,12,None,0.000,0,0,0.000,0,0.000,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2020-12-05,293,8326068082,1,0,8015042.5369619169,0.000,0,0,0.000,0.000,1,None,0.000,1,0,0.000,0,0.000,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2020-12-07,0,9688417034,1,1,5786172.1537126919,0.000,0,0,0.000,0.000,0,None,0.000,0,0,0.000,0,0.000,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0

In [ ]:
# check null values
df.isna().sum()

event_date                                                0
event_params_engagement_time_msec                         0
event_params_ga_session_id                                0
event_params_ga_session_number                            0
event_params_session_engaged                              0
user_pseudo_id                                            0
user_ltv_revenue                                          0
ecommerce_total_item_quantity                             0
total_return_item_quantity                                0
ecommerce_purchase_revenue_in_usd                         0
ecommerce_tax_value_in_usd                                0
ecommerce_unique_items                                    0
ecommerce_transaction_id                            1987843
item_price_in_usd                                         0
item_promotion_name                                       0
item_quantity                                             0
item_refund_in_usd                      

In [ ]:
# check number of unique order IDs (ecommerce_transaction_id)
print(df['ecommerce_transaction_id'].nunique())

4451


In [ ]:
# check unique combinations of user_pseudo_id and ecommerce_transaction_id
unique_combos = df.groupby(['user_pseudo_id', 'ecommerce_transaction_id']).size().reset_index(name='count').shape[0]
print(unique_combos)

4466


In [ ]:
unique_combos = df.groupby(['user_pseudo_id', 'event_params_ga_session_id']).size().reset_index(name='count').shape[0]
print(unique_combos)

15760


In [ ]:
# sort the data by user_pseudo_id, event_date, event_params_ga_session_number
df_sorted = df.sort_values(by=['user_pseudo_id', 'event_date', 'event_params_ga_session_number'])

### Aggregating the data

Aggregate transaction-related values separately from browsing session data.

**Transaction-related columns:** <br>
Aggregate these values only for valid transactions (ecommerce_transaction_id is not null)

**Columns to group by:**
1. 'user_pseudo_id'
2. 'event_params_ga_session_id'
3. 'event_params_ga_session_number'
4. 'ecommerce_transaction_id'

**Columns to sum:**
- 'item_price_in_usd'
- 'item_promotion_name'
- 'item_quantity'
- 'item_refund_in_usd'
- 'item_refund_quantity'
- 'item_revenue_in_usd'

**Columns to average:**
- 'item_price_in_usd'
- 'item_promotion_name'
- 'item_quantity'
- 'item_refund_in_usd'
- 'item_refund_quantity'
- 'item_revenue_in_usd'

**Columns to get max:**
- 'item_price_in_usd'
- 'item_promotion_name'
- 'item_quantity'
- 'item_refund_in_usd'
- 'item_refund_quantity'
- 'item_revenue_in_usd'

**Columns to de-duplicate:**
- **'user_ltv_revenue'**
    - First find the max of each transaction to get one value per transaction.
- 'ecommerce_total_item_quantity'
- 'total_return_item_quantity'
- 'ecommerce_purchase_revenue_in_usd'
- **'ecommerce_refund_value_in_usd'**
    - Create this column based on the sum of 'item_refund_in_usd'.
- 'ecommerce_tax_value_in_usd'
- 'ecommerce_unique_items'

**Session-related columns:** <br>
Aggregate all rows related to the same session. <br>
Later, these sessions will be split into pre- and during the transaction-session sessions.

**Columns to group by:**
1. 'user_pseudo_id'
2. 'event_params_ga_session_id'
3. 'event_params_ga_session_number'
4. 'ecommerce_transaction_id'

**Columns to de-duplicate:** <br>
These features are constant across a session.
- **'event_date'**
    - After identifying on what date a majority of the session took place.
- 'device_category_desktop'
- 'device_category_mobile'
- 'device_mobile_brand_name_Apple'
- 'device_mobile_brand_name_Google'
- 'device_mobile_brand_name_Huawei'
- 'device_mobile_brand_name_Microsoft'
- 'device_mobile_brand_name_Mozilla'
- 'device_mobile_brand_name_Samsung'
- 'device_mobile_brand_name_Xiaomi'
- 'device_mobile_model_name_Chrome'
- 'device_mobile_model_name_ChromeBook'
- 'device_mobile_model_name_Edge'
- 'device_mobile_model_name_Firefox'
- 'device_mobile_model_name_Safari'
- 'device_mobile_model_name_iPad'
- 'device_mobile_model_name_iPhone'
- 'device_web_info_browser_AndroidWebview'
- 'device_web_info_browser_Chrome'
- 'device_web_info_browser_Edge'
- 'device_web_info_browser_Firefox'
- 'device_web_info_browser_Safari'
- 'geo_country_Canada'
- 'geo_country_France'
- 'geo_country_India'
- 'geo_country_Other'
- 'geo_country_Spain'
- 'geo_country_UnitedKingdom'
- 'geo_country_UnitedStates'
- 'traffic_source_medium_Other'
- 'traffic_source_medium_cpc'
- 'traffic_source_medium_organic'
- 'traffic_source_medium_referral'

**Columns to average:**
- 'event_params_engagement_time_msec'

**Columns to get max:**
- 'event_params_engagement_time_msec'

**Columns to sum:**
- 'event_params_engagement_time_msec'
- 'event_params_session_engaged'
- 'item_promotion_name'
- 'event_name_add_to_cart'
- 'event_name_begin_checkout'
- 'event_name_page_view'
- 'event_name_scroll'
- 'event_name_select_item'
- 'event_name_user_engagement'
- 'event_name_view_item'
- 'event_params_parent_page_Apparel'
- 'event_params_parent_page_CampusCollection'
- 'event_params_parent_page_CheckoutConfirmation'
- 'event_params_parent_page_CheckoutYourInformation'
- 'event_params_parent_page_EcoFriendly'
- 'event_params_parent_page_Home'
- 'event_params_parent_page_Lifestyle'
- 'event_params_parent_page_New'
- 'event_params_parent_page_Other'
- 'event_params_parent_page_PaymentMethod'
- 'event_params_parent_page_Sale'
- 'event_params_parent_page_ShopbyBrand'
- 'event_params_parent_page_ShoppingCart'
- 'event_params_parent_page_Stationery'
- 'event_params_child_page_Bags'
- 'event_params_child_page_Drinkware'
- 'event_params_child_page_Google'
- 'event_params_child_page_Hats'
- 'event_params_child_page_Kids'
- 'event_params_child_page_MensUnisex'
- 'event_params_child_page_Notebooks'
- 'event_params_child_page_Other'
- 'event_params_child_page_SmallGoods'
- 'event_params_child_page_Socks'
- 'event_params_child_page_Stickers'
- 'event_params_child_page_Womens'
- 'event_params_child_page_Writing'
- 'event_params_child_page_YouTube'
- 'item_parent_category_Apparel'
- 'item_parent_category_Collections'
- 'item_parent_category_Lifestyle'
- 'item_parent_category_New'
- 'item_parent_category_Other'
- 'item_parent_category_Sale'
- 'item_parent_category_ShopbyBrand'
- 'item_parent_category_Stationery'
- 'item_child_category_Bags'
- 'item_child_category_CampusCollection'
- 'item_child_category_Drinkware'
- 'item_child_category_Google'
- 'item_child_category_Kids'
- 'item_child_category_MensUnisex'
- 'item_child_category_Other'
- 'item_child_category_SmallGoods'
- 'item_child_category_Womens'
- 'item_child_subcategory_Backpacks'
- 'item_child_subcategory_ElectronicsAccessories'
- 'item_child_subcategory_Infant'
- 'item_child_subcategory_MensTShirts'
- 'item_child_subcategory_MugsTumblers'
- 'item_child_subcategory_Other'
- 'item_child_subcategory_WaterBottles'

### Transaction Aggregation
To uniquely identifiy a transaction and be able to join this information with a particular session later, use `user_pseudo_id`, `event_params_ga_session_id`, `event_params_ga_session_number`, and `ecommerce_transaction_id` to group by.

In [ ]:
# exclude rows where ecommerce_transaction_id is null
valid_transactions = df_sorted[df_sorted['ecommerce_transaction_id'].notna()]
valid_transactions.shape

(14167, 110)

In [ ]:
valid_transactions[[
    'user_pseudo_id',
    'event_params_ga_session_id',
    'event_params_ga_session_number',
    'ecommerce_transaction_id'
]].drop_duplicates().shape

(4466, 4)

The number of unique identifiers in `valid_transactions` matches our expected value for unique sessions with a transaction. <br>
Let's start by aggregating the item-level quantities and monetary values to the transaction level.

#### Sum-item features

In [ ]:
transaction_sum = valid_transactions.groupby(['user_pseudo_id', 'event_params_ga_session_id', 'event_params_ga_session_number', 'ecommerce_transaction_id'])[
    'item_price_in_usd',
    'item_promotion_name',
    'item_quantity',
    'item_refund_in_usd',
    'item_refund_quantity',
    'item_revenue_in_usd'
].agg('sum').reset_index()

In [ ]:
# update column names to reflect the aggregation performed on them
transaction_sum.columns = ['user_pseudo_id',
                           'event_params_ga_session_id',
                           'event_params_ga_session_number',
                           'ecommerce_transaction_id',
                           'sum_item_price_in_usd',
                           'sum_item_promotions',
                           'sum_item_quantity',
                           'sum_item_refund_in_usd',
                           'sum_item_refund_quantity',
                           'sum_item_revenue_in_usd']

#### Average-item features

In [ ]:
transaction_avg = valid_transactions.groupby(['user_pseudo_id', 'event_params_ga_session_id', 'event_params_ga_session_number', 'ecommerce_transaction_id'])[
    'item_price_in_usd',
    'item_promotion_name',
    'item_quantity',
    'item_refund_in_usd',
    'item_refund_quantity',
    'item_revenue_in_usd'
].agg('mean').reset_index()

In [ ]:
# update column names to reflect the aggregation performed on them
transaction_avg.columns = ['user_pseudo_id',
                           'event_params_ga_session_id',
                           'event_params_ga_session_number',
                           'ecommerce_transaction_id',
                           'avg_item_price_in_usd',
                           'avg_item_promotions',
                           'avg_item_quantity',
                           'avg_item_refund_in_usd',
                           'avg_item_refund_quantity',
                           'avg_item_revenue_in_usd']

#### Max-item features

In [ ]:
transaction_max = valid_transactions.groupby(['user_pseudo_id', 'event_params_ga_session_id', 'event_params_ga_session_number', 'ecommerce_transaction_id'])[
    'item_price_in_usd',
    'item_promotion_name',
    'item_quantity',
    'item_refund_in_usd',
    'item_refund_quantity',
    'item_revenue_in_usd'
].agg('max').reset_index()

In [ ]:
# update column names to reflect the aggregation performed on them
transaction_max.columns = ['user_pseudo_id',
                           'event_params_ga_session_id',
                           'event_params_ga_session_number',
                           'ecommerce_transaction_id',
                           'max_item_price_in_usd',
                           'max_item_promotions',
                           'max_item_quantity',
                           'max_item_refund_in_usd',
                           'max_item_refund_quantity',
                           'max_item_revenue_in_usd']

#### `ecommerce_refund_value_in_usd`

In [ ]:
# create the ecommerce_refund_value_in_usd column as sum(item_refund_in_usd)
transaction_refund = transaction_sum[[
    'user_pseudo_id',
    'event_params_ga_session_id',
    'event_params_ga_session_number',
    'ecommerce_transaction_id',
    'sum_item_refund_in_usd'
]]

transaction_refund.columns = ['user_pseudo_id',
                              'event_params_ga_session_id',
                              'event_params_ga_session_number',
                              'ecommerce_transaction_id',
                              'ecommerce_refund_value_in_usd']

valid_transactions = pd.merge(valid_transactions, transaction_refund, how='left', on=['user_pseudo_id',
                                                                                      'event_params_ga_session_id',
                                                                                      'event_params_ga_session_number',
                                                                                      'ecommerce_transaction_id'])

In [ ]:
# sanity check on full returns
valid_transactions[(valid_transactions['total_return_item_quantity'] == valid_transactions['ecommerce_total_item_quantity'])
                   & (valid_transactions['ecommerce_purchase_revenue_in_usd'] != valid_transactions['ecommerce_refund_value_in_usd'])].shape

(931, 111)

There is a non-zero number of full returns that the sum of items refunded does not equal the original purchase revenue. In most cases, this is only a $1-2 difference, so simply replace the `ecommerce_refund_value_in_usd` with the `ecommerce_purchase_revenue_in_usd` in these cases.

In [ ]:
valid_transactions['ecommerce_refund_value_in_usd'] = valid_transactions.apply(lambda x: x['ecommerce_purchase_revenue_in_usd']
                                                                               if x['total_return_item_quantity'] == x['ecommerce_total_item_quantity']
                                                                               else x['ecommerce_refund_value_in_usd'],
                                                                               axis=1)

In [ ]:
# correction made
valid_transactions[(valid_transactions['total_return_item_quantity'] == valid_transactions['ecommerce_total_item_quantity'])
                   & (valid_transactions['ecommerce_purchase_revenue_in_usd'] != valid_transactions['ecommerce_refund_value_in_usd'])].shape

(0, 111)

#### `user_ltv_revenue`
For each transaction, let us associate each transaction with the largest `user_ltv_revenue` value recorded.

In [ ]:
transaction_ltv = valid_transactions.groupby(['user_pseudo_id', 'event_params_ga_session_id', 'event_params_ga_session_number', 'ecommerce_transaction_id'])[
    'user_ltv_revenue'
].agg('max').reset_index()

In [ ]:
# replace user_ltv_revenue with the max
valid_transactions = valid_transactions.drop('user_ltv_revenue', axis=1)
valid_transactions = pd.merge(valid_transactions, transaction_ltv, how='left', on=['user_pseudo_id',
                                                                                   'event_params_ga_session_id',
                                                                                   'event_params_ga_session_number',
                                                                                   'ecommerce_transaction_id'])

#### De-duplicate transaction features
Now that the transaction-level values are all present, de-duplicate the `ecommerce` features.

In [ ]:
transaction_unique = valid_transactions[[
    'user_pseudo_id',
    'event_params_ga_session_id',
    'event_params_ga_session_number',
    'ecommerce_transaction_id',
    'user_ltv_revenue',
    'ecommerce_total_item_quantity',
    'total_return_item_quantity',
    'ecommerce_purchase_revenue_in_usd',
    'ecommerce_refund_value_in_usd',
    'ecommerce_tax_value_in_usd',
    'ecommerce_unique_items'
]].drop_duplicates()

In [ ]:
transaction_unique.shape

(4466, 11)

### Join all transaction-aggregated features
Merge all parts of the transaction back together

In [ ]:
dfs = [transaction_unique, transaction_sum, transaction_avg, transaction_max]

# Define the columns on which to merge
merge_cols = ['user_pseudo_id', 'event_params_ga_session_id', 'event_params_ga_session_number', 'ecommerce_transaction_id']

# Use reduce to merge the DataFrames one by one
df_transaction = reduce(lambda left, right: pd.merge(left, right, on=merge_cols), dfs)

In [ ]:
df_transaction.shape

(4466, 29)

In [ ]:
df_transaction.head(10)

,user_pseudo_id,event_params_ga_session_id,event_params_ga_session_number,ecommerce_transaction_id,user_ltv_revenue,ecommerce_total_item_quantity,total_return_item_quantity,ecommerce_purchase_revenue_in_usd,ecommerce_refund_value_in_usd,ecommerce_tax_value_in_usd,ecommerce_unique_items,sum_item_price_in_usd,sum_item_promotions,sum_item_quantity,sum_item_refund_in_usd,sum_item_refund_quantity,sum_item_revenue_in_usd,avg_item_price_in_usd,avg_item_promotions,avg_item_quantity,avg_item_refund_in_usd,avg_item_refund_quantity,avg_item_revenue_in_usd,max_item_price_in_usd,max_item_promotions,max_item_quantity,max_item_refund_in_usd,max_item_refund_quantity,max_item_revenue_in_usd
0,10092926.3786306416,8014902521,1,719410,90.000,2,0,90.000,0.000,0.000,2,89.000,0,2,0.000,0,89.000,44.500,0.000,1.000,0.000,0.000,44.500,70.000,0,1,0.000,0,70.000
1,10111055.8768683862,6975932005,1,741471,94.000,3,0,94.000,0.000,10.000,3,94.000,0,3,0.000,0,94.000,31.333,0.000,1.000,0.000,0.000,31.333,48.000,0,1,0.000,0,48.000
2,1016446.8237887674,2633554037,2,983645,86.000,6,0,86.000,0.000,8.000,6,85.000,0,6,0.000,0,85.000,14.167,0.000,1.000,0.000,0.000,14.167,24.000,0,1,0.000,0,24.000
3,10172849.5375293351,4974585843,1,406646,156.000,6,6,156.000,156.000,15.000,6,155.000,0,6,155.000,6,155.000,25.833,0.000,1.000,25.833,1.000,25.833,48.000,0,1,48.000,1,48.000
4,1019527.5799124267,9903438389,1,2105,44.000,7,0,44.000,0.000,5.000,4,32.000,0,7,0.000,0,44.000,8.000,0.000,1.750,0.000,0.000,11.000,13.000,0,4,0.000,0,16.000
5,10211474.7929477742,4855749522,10,886501,29.000,1,1,10.000,10.000,1.000,1,10.000,0,1,10.000,1,10.000,10.000,0.000,1.000,10.000,1.000,10.000,10.000,0,1,10.000,1,10.000
6,1021887.1151788384,3688562872,1,76937,21.000,2,0,21.000,0.000,2.000,1,10.000,1,2,0.000,0,21.000,10.000,1.000,2.000,0.000,0.000,21.000,10.000,1,2,0.000,0,21.000
7,10269098.5703813861,5196652640,2,877956,66.000,22,0,66.000,0.000,5.000,1,3.000,1,22,0.000,0,66.000,3.000,1.000,22.000,0.000,0.000,66.000,3.000,1,22,0.000,0,66.000
8,1026932.0858862293,8144402352,1,339943,110.000,1,0,55.000,0.000,4.000,1,110.000,2,2,0.000,0,110.000,55.000,1.000,1.000,0.000,0.000,55.000,55.000,1,1,0.000,0,55.000
9,1026932.0858862293,4640172405,3,614046,158.000,1,0,48.000,0.000,4.000,1,48.000,0,1,0.000,0,48.000,48.000,0.000,1.000,0.000,0.000,48.000,48.000,0,1,0.000,0,48.000


In [ ]:
############################################################
# CHECKPOINT - save table to BQ to save progress if needed #
############################################################

client = bigquery.Client(project = project_name)
table_id = 'return_prediction_ga4.step_3_transactions_merged' # update your dataset id here as needed

job_config = bigquery.LoadJobConfig(
    write_disposition = bigquery.job.WriteDisposition.WRITE_TRUNCATE
)

job = client.load_table_from_dataframe(
    df_transaction, table_id, job_config = job_config
)

# Wait for the load job to complete.
job.result()

LoadJob<project=adl-analytics, location=US, id=970a4610-8ced-4cfc-8bbc-bbddb42fc441>

### Session Aggregation
To uniquely identify a session, we should use a combination of `user_pseudo_id`, `event_params_ga_session_id`, `event_params_ga_session_number` to group by.

#### De-duplicate session features

In [ ]:
# de-deuplicate session-constant features
session_constant = df_sorted[[
    'user_pseudo_id',
    'event_params_ga_session_id',
    'event_params_ga_session_number',
    'event_date',
    'device_category_desktop',
    'device_category_mobile',
    'device_mobile_brand_name_Apple',
    'device_mobile_brand_name_Google',
    'device_mobile_brand_name_Huawei',
    'device_mobile_brand_name_Microsoft',
    'device_mobile_brand_name_Mozilla',
    'device_mobile_brand_name_Samsung',
    'device_mobile_brand_name_Xiaomi',
    'device_mobile_model_name_Chrome',
    'device_mobile_model_name_ChromeBook',
    'device_mobile_model_name_Edge',
    'device_mobile_model_name_Firefox',
    'device_mobile_model_name_Safari',
    'device_mobile_model_name_iPad',
    'device_mobile_model_name_iPhone',
    'device_web_info_browser_AndroidWebview',
    'device_web_info_browser_Chrome',
    'device_web_info_browser_Edge',
    'device_web_info_browser_Firefox',
    'device_web_info_browser_Safari',
    'geo_country_Canada',
    'geo_country_France',
    'geo_country_India',
    'geo_country_Other',
    'geo_country_Spain',
    'geo_country_UnitedKingdom',
    'geo_country_UnitedStates',
    'traffic_source_medium_Other',
    'traffic_source_medium_cpc',
    'traffic_source_medium_organic',
    'traffic_source_medium_referral'
]].drop_duplicates()

In [ ]:
session_constant.shape

(15882, 36)

The number of rows in df_key does not match our original count for number of unique sessions. This is because a session could take place over multiple days, therefore, we should adjust the event_date to be the same across the entire session.

I'll use the mode of event_date for the entire session, assigning the date where a majority of the browsing occurred.

In [ ]:
session_date = df_sorted.groupby(['user_pseudo_id', 'event_params_ga_session_id', 'event_params_ga_session_number'])[
    'event_date'
].agg(pd.Series.mode).reset_index()

In [ ]:
session_date.shape

(15760, 4)

In [ ]:
# replace event_date with the mode of event date for each session
df_sorted = df_sorted.drop('event_date', axis=1)
df_sorted = pd.merge(df_sorted, session_date, how='left', on=['user_pseudo_id', 'event_params_ga_session_id', 'event_params_ga_session_number'])

In [ ]:
# retry de-duplicating
session_constant = df_sorted[[
    'user_pseudo_id',
    'event_params_ga_session_id',
    'event_params_ga_session_number',
    'event_date',
    'device_category_desktop',
    'device_category_mobile',
    'device_mobile_brand_name_Apple',
    'device_mobile_brand_name_Google',
    'device_mobile_brand_name_Huawei',
    'device_mobile_brand_name_Microsoft',
    'device_mobile_brand_name_Mozilla',
    'device_mobile_brand_name_Samsung',
    'device_mobile_brand_name_Xiaomi',
    'device_mobile_model_name_Chrome',
    'device_mobile_model_name_ChromeBook',
    'device_mobile_model_name_Edge',
    'device_mobile_model_name_Firefox',
    'device_mobile_model_name_Safari',
    'device_mobile_model_name_iPad',
    'device_mobile_model_name_iPhone',
    'device_web_info_browser_AndroidWebview',
    'device_web_info_browser_Chrome',
    'device_web_info_browser_Edge',
    'device_web_info_browser_Firefox',
    'device_web_info_browser_Safari',
    'geo_country_Canada',
    'geo_country_France',
    'geo_country_India',
    'geo_country_Other',
    'geo_country_Spain',
    'geo_country_UnitedKingdom',
    'geo_country_UnitedStates',
    'traffic_source_medium_Other',
    'traffic_source_medium_cpc',
    'traffic_source_medium_organic',
    'traffic_source_medium_referral'
]].drop_duplicates()

In [ ]:
session_constant.shape

(15760, 36)

Now our total number of rows matches our expected number of unique sessions.

#### Average session features

In [ ]:
session_avg = df_sorted.groupby(['user_pseudo_id', 'event_params_ga_session_id', 'event_params_ga_session_number'])[
    'event_params_engagement_time_msec'
].agg('mean').reset_index()

In [ ]:
session_avg.columns = ['user_pseudo_id',
                       'event_params_ga_session_id',
                       'event_params_ga_session_number',
                       'avg_event_params_engagement_time_msec']

In [ ]:
session_avg.shape

(15760, 4)

In [ ]:
session_avg.describe()[['avg_event_params_engagement_time_msec']]

,avg_event_params_engagement_time_msec
count,15760.000
mean,7876.956
std,17695.780
min,0.000
25%,1809.513
50%,5753.048
75%,10298.327
max,1242241.000


#### Max session features

In [ ]:
session_max = df_sorted.groupby(['user_pseudo_id', 'event_params_ga_session_id', 'event_params_ga_session_number'])[
    'event_params_engagement_time_msec'
].agg('max').reset_index()

In [ ]:
session_max.columns = ['user_pseudo_id',
                       'event_params_ga_session_id',
                       'event_params_ga_session_number',
                       'max_event_params_engagement_time_msec']

In [ ]:
session_max.shape

(15760, 4)

In [ ]:
session_max.describe()[['max_event_params_engagement_time_msec']]

,max_event_params_engagement_time_msec
count,15760.000
mean,74613.334
std,321789.771
min,0.000
25%,7528.500
50%,38341.000
75%,85863.500
max,36399379.000


#### Sum session features

In [ ]:
session_sum = df_sorted.groupby(['user_pseudo_id', 'event_params_ga_session_id', 'event_params_ga_session_number'])[
    'event_params_engagement_time_msec',
    'event_params_session_engaged',
    'event_name_add_to_cart',
    'event_name_begin_checkout',
    'event_name_page_view',
    'event_name_scroll',
    'event_name_select_item',
    'event_name_user_engagement',
    'event_name_view_item',
    'event_params_parent_page_Apparel',
    'event_params_parent_page_CampusCollection',
    'event_params_parent_page_CheckoutConfirmation',
    'event_params_parent_page_CheckoutYourInformation',
    'event_params_parent_page_EcoFriendly',
    'event_params_parent_page_Home',
    'event_params_parent_page_Lifestyle',
    'event_params_parent_page_New',
    'event_params_parent_page_Other',
    'event_params_parent_page_PaymentMethod',
    'event_params_parent_page_Sale',
    'event_params_parent_page_ShopbyBrand',
    'event_params_parent_page_ShoppingCart',
    'event_params_parent_page_Stationery',
    'event_params_child_page_Bags',
    'event_params_child_page_Drinkware',
    'event_params_child_page_Google',
    'event_params_child_page_Hats',
    'event_params_child_page_Kids',
    'event_params_child_page_MensUnisex',
    'event_params_child_page_Notebooks',
    'event_params_child_page_Other',
    'event_params_child_page_SmallGoods',
    'event_params_child_page_Socks',
    'event_params_child_page_Stickers',
    'event_params_child_page_Womens',
    'event_params_child_page_Writing',
    'event_params_child_page_YouTube',
    'item_parent_category_Apparel',
    'item_parent_category_Collections',
    'item_parent_category_Lifestyle',
    'item_parent_category_New',
    'item_parent_category_Other',
    'item_parent_category_Sale',
    'item_parent_category_ShopbyBrand',
    'item_parent_category_Stationery',
    'item_child_category_Bags',
    'item_child_category_CampusCollection',
    'item_child_category_Drinkware',
    'item_child_category_Google',
    'item_child_category_Kids',
    'item_child_category_MensUnisex',
    'item_child_category_Other',
    'item_child_category_SmallGoods',
    'item_child_category_Womens',
    'item_child_subcategory_Backpacks',
    'item_child_subcategory_ElectronicsAccessories',
    'item_child_subcategory_Infant',
    'item_child_subcategory_MensTShirts',
    'item_child_subcategory_MugsTumblers',
    'item_child_subcategory_Other',
    'item_child_subcategory_WaterBottles'
].agg('sum').reset_index()

In [ ]:
session_sum.columns = ['user_pseudo_id',
                       'event_params_ga_session_id',
                       'event_params_ga_session_number',
                       'sum_event_params_engagement_time_msec',
                       'sum_event_params_session_engaged',
                       'sum_event_name_add_to_cart',
                       'sum_event_name_begin_checkout',
                       'sum_event_name_page_view',
                       'sum_event_name_scroll',
                       'sum_event_name_select_item',
                       'sum_event_name_user_engagement',
                       'sum_event_name_view_item',
                       'sum_event_params_parent_page_Apparel',
                       'sum_event_params_parent_page_CampusCollection',
                       'sum_event_params_parent_page_CheckoutConfirmation',
                       'sum_event_params_parent_page_CheckoutYourInformation',
                       'sum_event_params_parent_page_EcoFriendly',
                       'sum_event_params_parent_page_Home',
                       'sum_event_params_parent_page_Lifestyle',
                       'sum_event_params_parent_page_New',
                       'sum_event_params_parent_page_Other',
                       'sum_event_params_parent_page_PaymentMethod',
                       'sum_event_params_parent_page_Sale',
                       'sum_event_params_parent_page_ShopbyBrand',
                       'sum_event_params_parent_page_ShoppingCart',
                       'sum_event_params_parent_page_Stationery',
                       'sum_event_params_child_page_Bags',
                       'sum_event_params_child_page_Drinkware',
                       'sum_event_params_child_page_Google',
                       'sum_event_params_child_page_Hats',
                       'sum_event_params_child_page_Kids',
                       'sum_event_params_child_page_MensUnisex',
                       'sum_event_params_child_page_Notebooks',
                       'sum_event_params_child_page_Other',
                       'sum_event_params_child_page_SmallGoods',
                       'sum_event_params_child_page_Socks',
                       'sum_event_params_child_page_Stickers',
                       'sum_event_params_child_page_Womens',
                       'sum_event_params_child_page_Writing',
                       'sum_event_params_child_page_YouTube',
                       'sum_item_parent_category_Apparel',
                       'sum_item_parent_category_Collections',
                       'sum_item_parent_category_Lifestyle',
                       'sum_item_parent_category_New',
                       'sum_item_parent_category_Other',
                       'sum_item_parent_category_Sale',
                       'sum_item_parent_category_ShopbyBrand',
                       'sum_item_parent_category_Stationery',
                       'sum_item_child_category_Bags',
                       'sum_item_child_category_CampusCollection',
                       'sum_item_child_category_Drinkware',
                       'sum_item_child_category_Google',
                       'sum_item_child_category_Kids',
                       'sum_item_child_category_MensUnisex',
                       'sum_item_child_category_Other',
                       'sum_item_child_category_SmallGoods',
                       'sum_item_child_category_Womens',
                       'sum_item_child_subcategory_Backpacks',
                       'sum_item_child_subcategory_ElectronicsAccessories',
                       'sum_item_child_subcategory_Infant',
                       'sum_item_child_subcategory_MensTShirts',
                       'sum_item_child_subcategory_MugsTumblers',
                       'sum_item_child_subcategory_Other',
                       'sum_item_child_subcategory_WaterBottles']

In [ ]:
session_sum.shape

(15760, 64)

In [ ]:
session_sum.describe()

,event_params_ga_session_number,sum_event_params_engagement_time_msec,sum_event_params_session_engaged,sum_event_name_add_to_cart,sum_event_name_begin_checkout,sum_event_name_page_view,sum_event_name_scroll,sum_event_name_select_item,sum_event_name_user_engagement,sum_event_name_view_item,sum_event_params_parent_page_Apparel,sum_event_params_parent_page_CampusCollection,sum_event_params_parent_page_CheckoutConfirmation,sum_event_params_parent_page_CheckoutYourInformation,sum_event_params_parent_page_EcoFriendly,sum_event_params_parent_page_Home,sum_event_params_parent_page_Lifestyle,sum_event_params_parent_page_New,sum_event_params_parent_page_Other,sum_event_params_parent_page_PaymentMethod,sum_event_params_parent_page_Sale,sum_event_params_parent_page_ShopbyBrand,sum_event_params_parent_page_ShoppingCart,sum_event_params_parent_page_Stationery,sum_event_params_child_page_Bags,sum_event_params_child_page_Drinkware,sum_event_params_child_page_Google,sum_event_params_child_page_Hats,sum_event_params_child_page_Kids,sum_event_params_child_page_MensUnisex,sum_event_params_child_page_Notebooks,sum_event_params_child_page_Other,sum_event_params_child_page_SmallGoods,sum_event_params_child_page_Socks,sum_event_params_child_page_Stickers,sum_event_params_child_page_Womens,sum_event_params_child_page_Writing,sum_event_params_child_page_YouTube,sum_item_parent_category_Apparel,sum_item_parent_category_Collections,sum_item_parent_category_Lifestyle,sum_item_parent_category_New,sum_item_parent_category_Other,sum_item_parent_category_Sale,sum_item_parent_category_ShopbyBrand,sum_item_parent_category_Stationery,sum_item_child_category_Bags,sum_item_child_category_CampusCollection,sum_item_child_category_Drinkware,sum_item_child_category_Google,sum_item_child_category_Kids,sum_item_child_category_MensUnisex,sum_item_child_category_Other,sum_item_child_category_SmallGoods,sum_item_child_category_Womens,sum_item_child_subcategory_Backpacks,sum_item_child_subcategory_ElectronicsAccessories,sum_item_child_subcategory_Infant,sum_item_child_subcategory_MensTShirts,sum_item_child_subcategory_MugsTumblers,sum_item_child_subcategory_Other,sum_item_child_subcategory_WaterBottles
count,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000,15760.000
mean,4.569,1338443.038,124.864,19.002,3.853,14.488,7.165,6.726,13.291,57.546,34.993,2.731,1.915,6.646,2.169,15.819,15.769,3.206,1.959,2.594,12.667,5.174,5.280,5.702,3.130,6.152,2.899,2.904,3.384,17.688,1.963,0.812,4.209,0.623,0.902,6.585,1.349,1.093,36.218,3.515,17.575,2.993,0.230,14.740,5.509,5.235,2.996,3.514,6.158,3.213,3.411,18.113,10.672,3.832,6.509,0.003,0.013,0.007,0.113,0.042,0.003,0.004
std,5.223,2600054.114,206.846,54.303,11.392,19.543,9.804,21.283,17.976,106.192,80.607,19.181,3.505,15.420,12.225,52.405,45.410,12.695,6.157,5.059,48.653,27.765,10.013,25.092,15.007,25.746,22.029,14.175,19.385,52.196,12.910,8.864,19.204,3.665,6.315,28.461,10.656,10.402,84.106,22.682,50.272,12.328,1.901,56.973,30.518,24.614,15.158,22.682,25.939,24.999,20.452,53.737,33.860,18.610,28.792,0.257,0.303,0.284,3.339,1.516,0.231,0.267
min,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,

### Join all session-aggregated features
Merge all parts of the session back together

In [ ]:
# check shape of each first to make sure nothing is unexpected
# we want to make sure the row count is the same across all dfs
print(session_constant.shape)
print(session_avg.shape)
print(session_max.shape)
print(session_sum.shape)

(15760, 36)
(15760, 4)
(15760, 4)
(15760, 64)


In [ ]:
dfs = [session_constant, session_avg, session_max, session_sum]

# Define the columns on which to merge
merge_cols = ['user_pseudo_id', 'event_params_ga_session_id', 'event_params_ga_session_number']

# Use reduce to merge the DataFrames one by one
df_session = reduce(lambda left, right: pd.merge(left, right, on=merge_cols), dfs)

In [ ]:
df_session.shape

(15760, 99)

In [ ]:
############################################################
# CHECKPOINT - save table to BQ to save progress if needed #
############################################################

client = bigquery.Client(project = project_name)
table_id = 'return_prediction_ga4.step_3_session_merged' # update your dataset id here as needed

job_config = bigquery.LoadJobConfig(
    write_disposition = bigquery.job.WriteDisposition.WRITE_TRUNCATE
)

job = client.load_table_from_dataframe(
    df_session, table_id, job_config = job_config
)

# Wait for the load job to complete.
job.result()

LoadJob<project=adl-analytics, location=US, id=df841f30-b220-4fe9-b1f1-b26a9ae0d850>

## Put transaction and session back together

In [ ]:
df_merged = pd.merge(df_session, df_transaction, how='left', on=['user_pseudo_id', 'event_params_ga_session_id', 'event_params_ga_session_number'])

In [ ]:
df_merged.shape

(16182, 125)

After merging transactions, we end up with more rows than original sessions, since a session can contain multiple transactions.

In [ ]:
df_sorted = df_merged.sort_values(by=[
    'user_pseudo_id',
    'event_date',
    'event_params_ga_session_number',
    'ecommerce_transaction_id'
    ])

In [ ]:
# fill null values in ecommerce_transaction_id to allow groupby on that field
df_sorted['ecommerce_transaction_id'] = df_sorted['ecommerce_transaction_id'].fillna('not_applicable')

In [ ]:
# filter df only for users who have made at least one purchase
user_ids_to_include = df_sorted[df_sorted['ecommerce_transaction_id'] != 'not_applicable'].groupby('user_pseudo_id').size().index

# Filter the DataFrame to exclude user_pseudo_ids with all 'not_applicable' values
df_filtered = df_sorted[df_sorted['user_pseudo_id'].isin(user_ids_to_include)]

In [ ]:
df_filtered.shape

(13839, 125)

In [ ]:
############################################################
# CHECKPOINT - save table to BQ to save progress if needed #
############################################################

client = bigquery.Client(project = project_name)
table_id = 'return_prediction_ga4.step_3_filtered' # update your dataset id here as needed

job_config = bigquery.LoadJobConfig(
    write_disposition = bigquery.job.WriteDisposition.WRITE_TRUNCATE
)

job = client.load_table_from_dataframe(
    df_filtered, table_id, job_config = job_config
)

# Wait for the load job to complete.
job.result()

LoadJob<project=adl-analytics, location=US, id=983766c4-b87f-4cf1-85f1-ea08e15cbf8b>

## Split pre- & during-transaction sessions and aggregate

In [ ]:
df_filtered = df_filtered.sort_values(by=[
    'user_pseudo_id',
    'event_date',
    'ecommerce_transaction_id'
], ascending=[True, True, False])

In [ ]:
# in the above, add a label Y/N called "transaction" and get rid of rows that occurred after the last transaction date that don't have a following transaction
df_filtered['transaction_id_present'] = 'Y' # initialize with Y
df_filtered.loc[df_filtered['ecommerce_transaction_id'] == 'not_applicable', 'transaction_id_present'] = 'N' # replace not_applicable rows with N

In [ ]:
# replace not_applicable with NaN
df_filtered['ecommerce_transaction_id'] = df_filtered['ecommerce_transaction_id'].replace('not_applicable', np.nan)

In [ ]:
df_filtered['ecommerce_transaction_id'] = df_filtered.groupby(['user_pseudo_id', 'event_params_ga_session_id'])['ecommerce_transaction_id'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))


In [ ]:
df_filtered['ecommerce_transaction_id'] = df_filtered['ecommerce_transaction_id'].fillna('not_applicable')

In [ ]:
columns_to_sum = [
    'sum_event_params_engagement_time_msec',
    'sum_event_params_session_engaged',
    'user_ltv_revenue',
    'ecommerce_total_item_quantity',
    'total_return_item_quantity',
    'ecommerce_purchase_revenue_in_usd',
    'ecommerce_refund_value_in_usd',
    'ecommerce_tax_value_in_usd',
    'ecommerce_unique_items',
    'sum_item_price_in_usd',
    'sum_item_promotions',
    'sum_item_quantity',
    'sum_item_refund_in_usd',
    'sum_item_refund_quantity',
    'sum_item_revenue_in_usd',
    'sum_event_name_add_to_cart',
    'sum_event_name_begin_checkout',
    'sum_event_name_page_view',
    'sum_event_name_scroll',
    'sum_event_name_select_item',
    'sum_event_name_user_engagement',
    'sum_event_name_view_item',
    'device_category_desktop',
    'device_category_mobile',
    'device_mobile_brand_name_Apple',
    'device_mobile_brand_name_Google',
    'device_mobile_brand_name_Huawei',
    'device_mobile_brand_name_Microsoft',
    'device_mobile_brand_name_Mozilla',
    'device_mobile_brand_name_Samsung',
    'device_mobile_brand_name_Xiaomi',
    'device_mobile_model_name_Chrome',
    'device_mobile_model_name_ChromeBook',
    'device_mobile_model_name_Edge',
    'device_mobile_model_name_Firefox',
    'device_mobile_model_name_Safari',
    'device_mobile_model_name_iPad',
    'device_mobile_model_name_iPhone',
    'device_web_info_browser_AndroidWebview',
    'device_web_info_browser_Chrome',
    'device_web_info_browser_Edge',
    'device_web_info_browser_Firefox',
    'device_web_info_browser_Safari',
    'geo_country_Canada',
    'geo_country_France',
    'geo_country_India',
    'geo_country_Other',
    'geo_country_Spain',
    'geo_country_UnitedKingdom',
    'geo_country_UnitedStates',
    'traffic_source_medium_Other',
    'traffic_source_medium_cpc',
    'traffic_source_medium_organic',
    'traffic_source_medium_referral',
    'sum_event_params_parent_page_Apparel',
    'sum_event_params_parent_page_CampusCollection',
    'sum_event_params_parent_page_CheckoutConfirmation',
    'sum_event_params_parent_page_CheckoutYourInformation',
    'sum_event_params_parent_page_EcoFriendly',
    'sum_event_params_parent_page_Home',
    'sum_event_params_parent_page_Lifestyle',
    'sum_event_params_parent_page_New',
    'sum_event_params_parent_page_Other',
    'sum_event_params_parent_page_PaymentMethod',
    'sum_event_params_parent_page_Sale',
    'sum_event_params_parent_page_ShopbyBrand',
    'sum_event_params_parent_page_ShoppingCart',
    'sum_event_params_parent_page_Stationery',
    'sum_event_params_child_page_Bags',
    'sum_event_params_child_page_Drinkware',
    'sum_event_params_child_page_Google',
    'sum_event_params_child_page_Hats',
    'sum_event_params_child_page_Kids',
    'sum_event_params_child_page_MensUnisex',
    'sum_event_params_child_page_Notebooks',
    'sum_event_params_child_page_Other',
    'sum_event_params_child_page_SmallGoods',
    'sum_event_params_child_page_Socks',
    'sum_event_params_child_page_Stickers',
    'sum_event_params_child_page_Womens',
    'sum_event_params_child_page_Writing',
    'sum_event_params_child_page_YouTube',
    'sum_item_parent_category_Apparel',
    'sum_item_parent_category_Collections',
    'sum_item_parent_category_Lifestyle',
    'sum_item_parent_category_New',
    'sum_item_parent_category_Other',
    'sum_item_parent_category_Sale',
    'sum_item_parent_category_ShopbyBrand',
    'sum_item_parent_category_Stationery',
    'sum_item_child_category_Bags',
    'sum_item_child_category_CampusCollection',
    'sum_item_child_category_Drinkware',
    'sum_item_child_category_Google',
    'sum_item_child_category_Kids',
    'sum_item_child_category_MensUnisex',
    'sum_item_child_category_Other',
    'sum_item_child_category_SmallGoods',
    'sum_item_child_category_Womens',
    'sum_item_child_subcategory_Backpacks',
    'sum_item_child_subcategory_ElectronicsAccessories',
    'sum_item_child_subcategory_Infant',
    'sum_item_child_subcategory_MensTShirts',
    'sum_item_child_subcategory_MugsTumblers',
    'sum_item_child_subcategory_Other',
    'sum_item_child_subcategory_WaterBottles'
]

# List of columns to get max
columns_to_max = [
    'max_event_params_engagement_time_msec',
    'max_item_price_in_usd',
    'max_item_promotions',
    'max_item_quantity',
    'max_item_refund_in_usd',
    'max_item_refund_quantity',
    'max_item_revenue_in_usd'
]

columns_to_avg = [
    'avg_event_params_engagement_time_msec',
    'avg_item_price_in_usd',
    'avg_item_promotions',
    'avg_item_quantity',
    'avg_item_refund_in_usd',
    'avg_item_refund_quantity',
    'avg_item_revenue_in_usd'
]

# Group by and aggregate
df_agg = df_filtered.groupby(['user_pseudo_id', 'event_date', 'event_params_ga_session_id', 'event_params_ga_session_number', 'ecommerce_transaction_id'])[
    columns_to_sum + columns_to_max + columns_to_avg
].agg({
    **{col: 'sum' for col in columns_to_sum},
    **{col: 'max' for col in columns_to_max},
    **{col: 'mean' for col in columns_to_avg}
}).reset_index()

In [ ]:
# rename constant session columns to sum
df_agg.columns = [
    'user_pseudo_id',
    'event_date',
    'event_params_ga_session_id',
    'event_params_ga_session_number',
    'ecommerce_transaction_id',
    'sum_event_params_engagement_time_msec',
    'sum_event_params_session_engaged',
    'user_ltv_revenue',
    'ecommerce_total_item_quantity',
    'total_return_item_quantity',
    'ecommerce_purchase_revenue_in_usd',
    'ecommerce_refund_value_in_usd',
    'ecommerce_tax_value_in_usd',
    'ecommerce_unique_items',
    'sum_item_price_in_usd',
    'sum_item_promotions',
    'sum_item_quantity',
    'sum_item_refund_in_usd',
    'sum_item_refund_quantity',
    'sum_item_revenue_in_usd',
    'sum_event_name_add_to_cart',
    'sum_event_name_begin_checkout',
    'sum_event_name_page_view',
    'sum_event_name_scroll',
    'sum_event_name_select_item',
    'sum_event_name_user_engagement',
    'sum_event_name_view_item',
    'sum_device_category_desktop',
    'sum_device_category_mobile',
    'sum_device_mobile_brand_name_Apple',
    'sum_device_mobile_brand_name_Google',
    'sum_device_mobile_brand_name_Huawei',
    'sum_device_mobile_brand_name_Microsoft',
    'sum_device_mobile_brand_name_Mozilla',
    'sum_device_mobile_brand_name_Samsung',
    'sum_device_mobile_brand_name_Xiaomi',
    'sum_device_mobile_model_name_Chrome',
    'sum_device_mobile_model_name_ChromeBook',
    'sum_device_mobile_model_name_Edge',
    'sum_device_mobile_model_name_Firefox',
    'sum_device_mobile_model_name_Safari',
    'sum_device_mobile_model_name_iPad',
    'sum_device_mobile_model_name_iPhone',
    'sum_device_web_info_browser_AndroidWebview',
    'sum_device_web_info_browser_Chrome',
    'sum_device_web_info_browser_Edge',
    'sum_device_web_info_browser_Firefox',
    'sum_device_web_info_browser_Safari',
    'sum_geo_country_Canada',
    'sum_geo_country_France',
    'sum_geo_country_India',
    'sum_geo_country_Other',
    'sum_geo_country_Spain',
    'sum_geo_country_UnitedKingdom',
    'sum_geo_country_UnitedStates',
    'sum_traffic_source_medium_Other',
    'sum_traffic_source_medium_cpc',
    'sum_traffic_source_medium_organic',
    'sum_traffic_source_medium_referral',
    'sum_event_params_parent_page_Apparel',
    'sum_event_params_parent_page_CampusCollection',
    'sum_event_params_parent_page_CheckoutConfirmation',
    'sum_event_params_parent_page_CheckoutYourInformation',
    'sum_event_params_parent_page_EcoFriendly',
    'sum_event_params_parent_page_Home',
    'sum_event_params_parent_page_Lifestyle',
    'sum_event_params_parent_page_New',
    'sum_event_params_parent_page_Other',
    'sum_event_params_parent_page_PaymentMethod',
    'sum_event_params_parent_page_Sale',
    'sum_event_params_parent_page_ShopbyBrand',
    'sum_event_params_parent_page_ShoppingCart',
    'sum_event_params_parent_page_Stationery',
    'sum_event_params_child_page_Bags',
    'sum_event_params_child_page_Drinkware',
    'sum_event_params_child_page_Google',
    'sum_event_params_child_page_Hats',
    'sum_event_params_child_page_Kids',
    'sum_event_params_child_page_MensUnisex',
    'sum_event_params_child_page_Notebooks',
    'sum_event_params_child_page_Other',
    'sum_event_params_child_page_SmallGoods',
    'sum_event_params_child_page_Socks',
    'sum_event_params_child_page_Stickers',
    'sum_event_params_child_page_Womens',
    'sum_event_params_child_page_Writing',
    'sum_event_params_child_page_YouTube',
    'sum_item_parent_category_Apparel',
    'sum_item_parent_category_Collections',
    'sum_item_parent_category_Lifestyle',
    'sum_item_parent_category_New',
    'sum_item_parent_category_Other',
    'sum_item_parent_category_Sale',
    'sum_item_parent_category_ShopbyBrand',
    'sum_item_parent_category_Stationery',
    'sum_item_child_category_Bags',
    'sum_item_child_category_CampusCollection',
    'sum_item_child_category_Drinkware',
    'sum_item_child_category_Google',
    'sum_item_child_category_Kids',
    'sum_item_child_category_MensUnisex',
    'sum_item_child_category_Other',
    'sum_item_child_category_SmallGoods',
    'sum_item_child_category_Womens',
    'sum_item_child_subcategory_Backpacks',
    'sum_item_child_subcategory_ElectronicsAccessories',
    'sum_item_child_subcategory_Infant',
    'sum_item_child_subcategory_MensTShirts',
    'sum_item_child_subcategory_MugsTumblers',
    'sum_item_child_subcategory_Other',
    'sum_item_child_subcategory_WaterBottles',
    'max_event_params_engagement_time_msec',
    'max_item_price_in_usd',
    'max_item_promotions',
    'max_item_quantity',
    'max_item_refund_in_usd',
    'max_item_refund_quantity',
    'max_item_revenue_in_usd',
    'avg_event_params_engagement_time_msec',
    'avg_item_price_in_usd',
    'avg_item_promotions',
    'avg_item_quantity',
    'avg_item_refund_in_usd',
    'avg_item_refund_quantity',
    'avg_item_revenue_in_usd'
]

In [ ]:
df_sorted = df_agg.sort_values(by=[
    'user_pseudo_id',
    'event_date',
    'ecommerce_transaction_id'
    ])

In [ ]:
df_sorted.head(10)

,user_pseudo_id,event_date,event_params_ga_session_id,event_params_ga_session_number,ecommerce_transaction_id,sum_event_params_engagement_time_msec,sum_event_params_session_engaged,user_ltv_revenue,ecommerce_total_item_quantity,total_return_item_quantity,ecommerce_purchase_revenue_in_usd,ecommerce_refund_value_in_usd,ecommerce_tax_value_in_usd,ecommerce_unique_items,sum_item_price_in_usd,sum_item_promotions,sum_item_quantity,sum_item_refund_in_usd,sum_item_refund_quantity,sum_item_revenue_in_usd,sum_event_name_add_to_cart,sum_event_name_begin_checkout,sum_event_name_page_view,sum_event_name_scroll,sum_event_name_select_item,sum_event_name_user_engagement,sum_event_name_view_item,sum_device_category_desktop,sum_device_category_mobile,sum_device_mobile_brand_name_Apple,sum_device_mobile_brand_name_Google,sum_device_mobile_brand_name_Huawei,sum_device_mobile_brand_name_Microsoft,sum_device_mobile_brand_name_Mozilla,sum_device_mobile_brand_name_Samsung,sum_device_mobile_brand_name_Xiaomi,sum_device_mobile_model_name_Chrome,sum_device_mobile_model_name_ChromeBook,sum_device_mobile_model_name_Edge,sum_device_mobile_model_name_Firefox,sum_device_mobile_model_name_Safari,sum_device_mobile_model_name_iPad,sum_device_mobile_model_name_iPhone,sum_device_web_info_browser_AndroidWebview,sum_device_web_info_browser_Chrome,sum_device_web_info_browser_Edge,sum_device_web_info_browser_Firefox,sum_device_web_info_browser_Safari,sum_geo_country_Canada,sum_geo_country_France,sum_geo_country_India,sum_geo_country_Other,sum_geo_country_Spain,sum_geo_country_UnitedKingdom,sum_geo_country_UnitedStates,sum_traffic_source_medium_Other,sum_traffic_source_medium_cpc,sum_traffic_source_medium_organic,sum_traffic_source_medium_referral,sum_event_params_parent_page_Apparel,sum_event_params_parent_page_CampusCollection,sum_event_params_parent_page_CheckoutConfirmation,sum_event_params_parent_page_CheckoutYourInformation,sum_event_params_parent_page_EcoFriendly,sum_event_params_parent_page_Home,sum_event_params_parent_page_Lifestyle,sum_event_params_parent_page_New,sum_event_params_parent_page_Other,sum_event_params_parent_page_PaymentMethod,sum_event_params_parent_page_Sale,sum_event_params_parent_page_ShopbyBrand,sum_event_params_parent_page_ShoppingCart,sum_event_params_parent_page_Stationery,sum_event_params_child_page_Bags,sum_event_params_child_page_Drinkware,sum_event_params_child_page_Google,sum_event_params_child_page_Hats,sum_event_params_child_page_Kids,sum_event_params_child_page_MensUnisex,sum_event_params_child_page_Notebooks,sum_event_params_child_page_Other,sum_event_params_child_page_SmallGoods,sum_event_params_child_page_Socks,sum_event_params_child_page_Stickers,sum_event_params_child_page_Womens,sum_event_params_child_page_Writing,sum_event_params_child_page_YouTube,sum_item_parent_category_Apparel,sum_item_parent_category_Collections,sum_item_parent_category_Lifestyle,sum_item_parent_category_New,sum_item_parent_category_Other,sum_item_parent_category_Sale,sum_item_parent_category_ShopbyBrand,sum_item_parent_category_Stationery,sum_item_child_category_Bags,sum_item_child_category_CampusCollection,sum_item_child_category_Drinkware,sum_item_child_category_Google,sum_item_child_category_Kids,sum_item_child_category_MensUnisex,sum_item_child_category_Other,sum_item_child_category_SmallGoods,sum_item_child_category_Womens,sum_item_child_subcategory_Backpacks,sum_item_child_subcategory_ElectronicsAccessories,sum_item_child_subcategory_Infant,sum_item_child_subcategory_MensTShirts,sum_item_child_subcategory_MugsTumblers,sum_item_child_subcategory_Other,sum_item_child_subcategory_WaterBottles,max_event_params_engagement_time_msec,max_item_price_in_usd,max_item_promotions,max_item_quantity,max_item_refund_in_usd,max_item_refund_quantity,max_item_revenue_in_usd,avg_event_params_engagement_time_msec,avg_item_price_in_usd,avg_item_promotions,avg_item_quantity,avg_item_refund_in_usd,avg_item_refund_quantity,avg_item_revenue_in_usd
1,10092926.378630641

In [ ]:
df_sorted.shape

(13839, 125)

In [ ]:
# remove user activity that occurred after the most recent transaction and does not have a transaction to group with
# Convert 'event_date' to datetime format if it's not already
df_sorted['event_date'] = pd.to_datetime(df_sorted['event_date'])

# Find the most recent date for each user_pseudo_id where ecommerce_transaction_id is not 'not_applicable'
recent_dates = df_sorted[df_sorted['ecommerce_transaction_id'] != 'not_applicable'].groupby('user_pseudo_id')['event_date'].max()

# Filter rows where 'ecommerce_transaction_id' is not 'not_applicable' or the event_date is on or before the most recent date
df_sorted2 = df_sorted[(df_sorted['ecommerce_transaction_id'] != 'not_applicable') | (df_sorted['event_date'] <= df_sorted['user_pseudo_id'].map(recent_dates))]

In [ ]:
df_sorted2.shape

(10737, 125)

In [ ]:
# create labels for session type
df_sorted2['session_type'] = 'transaction_session'
df_sorted2.loc[df_sorted2['ecommerce_transaction_id'] == 'not_applicable', 'session_type'] = 'pre_transaction_session'

In [ ]:
df_sorted2['session_type'].value_counts()

pre_transaction_session    6271
transaction_session        4466
Name: session_type, dtype: int64

In [ ]:
df_sorted2 = df_sorted2.sort_values(by=[
    'user_pseudo_id',
    'event_date',
    'ecommerce_transaction_id'
    ],ascending = [True, True, False])

In [ ]:
df_sorted2['ecommerce_transaction_id'] = df_sorted2['ecommerce_transaction_id'].replace('not_applicable', np.nan)

In [ ]:
df_sorted2.head()

,user_pseudo_id,event_date,event_params_ga_session_id,event_params_ga_session_number,ecommerce_transaction_id,sum_event_params_engagement_time_msec,sum_event_params_session_engaged,user_ltv_revenue,ecommerce_total_item_quantity,total_return_item_quantity,ecommerce_purchase_revenue_in_usd,ecommerce_refund_value_in_usd,ecommerce_tax_value_in_usd,ecommerce_unique_items,sum_item_price_in_usd,sum_item_promotions,sum_item_quantity,sum_item_refund_in_usd,sum_item_refund_quantity,sum_item_revenue_in_usd,sum_event_name_add_to_cart,sum_event_name_begin_checkout,sum_event_name_page_view,sum_event_name_scroll,sum_event_name_select_item,sum_event_name_user_engagement,sum_event_name_view_item,sum_device_category_desktop,sum_device_category_mobile,sum_device_mobile_brand_name_Apple,sum_device_mobile_brand_name_Google,sum_device_mobile_brand_name_Huawei,sum_device_mobile_brand_name_Microsoft,sum_device_mobile_brand_name_Mozilla,sum_device_mobile_brand_name_Samsung,sum_device_mobile_brand_name_Xiaomi,sum_device_mobile_model_name_Chrome,sum_device_mobile_model_name_ChromeBook,sum_device_mobile_model_name_Edge,sum_device_mobile_model_name_Firefox,sum_device_mobile_model_name_Safari,sum_device_mobile_model_name_iPad,sum_device_mobile_model_name_iPhone,sum_device_web_info_browser_AndroidWebview,sum_device_web_info_browser_Chrome,sum_device_web_info_browser_Edge,sum_device_web_info_browser_Firefox,sum_device_web_info_browser_Safari,sum_geo_country_Canada,sum_geo_country_France,sum_geo_country_India,sum_geo_country_Other,sum_geo_country_Spain,sum_geo_country_UnitedKingdom,sum_geo_country_UnitedStates,sum_traffic_source_medium_Other,sum_traffic_source_medium_cpc,sum_traffic_source_medium_organic,sum_traffic_source_medium_referral,sum_event_params_parent_page_Apparel,sum_event_params_parent_page_CampusCollection,sum_event_params_parent_page_CheckoutConfirmation,sum_event_params_parent_page_CheckoutYourInformation,sum_event_params_parent_page_EcoFriendly,sum_event_params_parent_page_Home,sum_event_params_parent_page_Lifestyle,sum_event_params_parent_page_New,sum_event_params_parent_page_Other,sum_event_params_parent_page_PaymentMethod,sum_event_params_parent_page_Sale,sum_event_params_parent_page_ShopbyBrand,sum_event_params_parent_page_ShoppingCart,sum_event_params_parent_page_Stationery,sum_event_params_child_page_Bags,sum_event_params_child_page_Drinkware,sum_event_params_child_page_Google,sum_event_params_child_page_Hats,sum_event_params_child_page_Kids,sum_event_params_child_page_MensUnisex,sum_event_params_child_page_Notebooks,sum_event_params_child_page_Other,sum_event_params_child_page_SmallGoods,sum_event_params_child_page_Socks,sum_event_params_child_page_Stickers,sum_event_params_child_page_Womens,sum_event_params_child_page_Writing,sum_event_params_child_page_YouTube,sum_item_parent_category_Apparel,sum_item_parent_category_Collections,sum_item_parent_category_Lifestyle,sum_item_parent_category_New,sum_item_parent_category_Other,sum_item_parent_category_Sale,sum_item_parent_category_ShopbyBrand,sum_item_parent_category_Stationery,sum_item_child_category_Bags,sum_item_child_category_CampusCollection,sum_item_child_category_Drinkware,sum_item_child_category_Google,sum_item_child_category_Kids,sum_item_child_category_MensUnisex,sum_item_child_category_Other,sum_item_child_category_SmallGoods,sum_item_child_category_Womens,sum_item_child_subcategory_Backpacks,sum_item_child_subcategory_ElectronicsAccessories,sum_item_child_subcategory_Infant,sum_item_child_subcategory_MensTShirts,sum_item_child_subcategory_MugsTumblers,sum_item_child_subcategory_Other,sum_item_child_subcategory_WaterBottles,max_event_params_engagement_time_msec,max_item_price_in_usd,max_item_promotions,max_item_quantity,max_item_refund_in_usd,max_item_refund_quantity,max_item_revenue_in_usd,avg_event_params_engagement_time_msec,avg_item_price_in_usd,avg_item_promotions,avg_item_quantity,avg_item_refund_in_usd,avg_item_refund_quantity,avg_item_revenue_in_usd,session_type
0,10092

In [ ]:
# backward fill ecommerce_transaction_id columns to group pre-transaction sessions with their respective transaction sessions
df_sorted2['ecommerce_transaction_id'] = df_sorted2.groupby('user_pseudo_id')['ecommerce_transaction_id'].bfill()

# now forward fill to capture the remaining NaNs that were on the same date as the transaction session
df_sorted2['ecommerce_transaction_id'] = df_sorted2.groupby('user_pseudo_id')['ecommerce_transaction_id'].ffill()

In [ ]:
############################################################
# CHECKPOINT - save table to BQ to save progress if needed #
############################################################

client = bigquery.Client(project = project_name)
table_id = 'return_prediction_ga4.step_3_transaction_ids_filled_ready_for_t_pt_split' # update your dataset id here as needed

job_config = bigquery.LoadJobConfig(
    write_disposition = bigquery.job.WriteDisposition.WRITE_TRUNCATE
)

job = client.load_table_from_dataframe(
    df_sorted2, table_id, job_config = job_config
)

# Wait for the load job to complete.
job.result()

LoadJob<project=adl-analytics, location=US, id=8b1978bf-fd1b-4670-b3c2-53221576fc6b>

In [ ]:
# split into two dfs: one for transaction sessions and one for pre-transaction sessions
df_t = df_sorted2[df_sorted2['session_type'] == 'transaction_session'] #transaction sessions
df_pt = df_sorted2[df_sorted2['session_type'] == 'pre_transaction_session'] # pre-transaction sessions

df_t = pd.DataFrame(df_t)
df_pt = pd.DataFrame(df_pt)

## Preparing the Transaction Session Table

In [ ]:
# check col and row count
print(df_t.shape)

(4466, 126)


In [ ]:
# count unique combos of user_pseudo_id, event_date and ecommerce_transaction_id
unique_combos_t = df_t.groupby(['user_pseudo_id', 'event_date','ecommerce_transaction_id']).size().reset_index(name='count').shape[0]
print(unique_combos_t)

4466


In [ ]:
# count unique combos of user_pseudo_id, event_date and ecommerce_transaction_id
unique_combos_t = df_t.groupby(['user_pseudo_id', 'ecommerce_transaction_id']).size().reset_index(name='count').shape[0]
print(unique_combos_t)

4466


In [ ]:
# drop event_params_session_id since it is no longer useful
df_t.drop('event_params_ga_session_id', axis=1, inplace=True)

In [ ]:
# it looks like there are cases where there is more than one date associated with a transaction id
# let's isolate those rows to take a closer look
duplicate_rows = df_t[df_t.duplicated(subset=['user_pseudo_id', 'ecommerce_transaction_id'], keep=False)]
print(duplicate_rows.shape)
duplicate_rows

(0, 125)


,user_pseudo_id,event_date,event_params_ga_session_number,ecommerce_transaction_id,sum_event_params_engagement_time_msec,sum_event_params_session_engaged,user_ltv_revenue,ecommerce_total_item_quantity,total_return_item_quantity,ecommerce_purchase_revenue_in_usd,ecommerce_refund_value_in_usd,ecommerce_tax_value_in_usd,ecommerce_unique_items,sum_item_price_in_usd,sum_item_promotions,sum_item_quantity,sum_item_refund_in_usd,sum_item_refund_quantity,sum_item_revenue_in_usd,sum_event_name_add_to_cart,sum_event_name_begin_checkout,sum_event_name_page_view,sum_event_name_scroll,sum_event_name_select_item,sum_event_name_user_engagement,sum_event_name_view_item,sum_device_category_desktop,sum_device_category_mobile,sum_device_mobile_brand_name_Apple,sum_device_mobile_brand_name_Google,sum_device_mobile_brand_name_Huawei,sum_device_mobile_brand_name_Microsoft,sum_device_mobile_brand_name_Mozilla,sum_device_mobile_brand_name_Samsung,sum_device_mobile_brand_name_Xiaomi,sum_device_mobile_model_name_Chrome,sum_device_mobile_model_name_ChromeBook,sum_device_mobile_model_name_Edge,sum_device_mobile_model_name_Firefox,sum_device_mobile_model_name_Safari,sum_device_mobile_model_name_iPad,sum_device_mobile_model_name_iPhone,sum_device_web_info_browser_AndroidWebview,sum_device_web_info_browser_Chrome,sum_device_web_info_browser_Edge,sum_device_web_info_browser_Firefox,sum_device_web_info_browser_Safari,sum_geo_country_Canada,sum_geo_country_France,sum_geo_country_India,sum_geo_country_Other,sum_geo_country_Spain,sum_geo_country_UnitedKingdom,sum_geo_country_UnitedStates,sum_traffic_source_medium_Other,sum_traffic_source_medium_cpc,sum_traffic_source_medium_organic,sum_traffic_source_medium_referral,sum_event_params_parent_page_Apparel,sum_event_params_parent_page_CampusCollection,sum_event_params_parent_page_CheckoutConfirmation,sum_event_params_parent_page_CheckoutYourInformation,sum_event_params_parent_page_EcoFriendly,sum_event_params_parent_page_Home,sum_event_params_parent_page_Lifestyle,sum_event_params_parent_page_New,sum_event_params_parent_page_Other,sum_event_params_parent_page_PaymentMethod,sum_event_params_parent_page_Sale,sum_event_params_parent_page_ShopbyBrand,sum_event_params_parent_page_ShoppingCart,sum_event_params_parent_page_Stationery,sum_event_params_child_page_Bags,sum_event_params_child_page_Drinkware,sum_event_params_child_page_Google,sum_event_params_child_page_Hats,sum_event_params_child_page_Kids,sum_event_params_child_page_MensUnisex,sum_event_params_child_page_Notebooks,sum_event_params_child_page_Other,sum_event_params_child_page_SmallGoods,sum_event_params_child_page_Socks,sum_event_params_child_page_Stickers,sum_event_params_child_page_Womens,sum_event_params_child_page_Writing,sum_event_params_child_page_YouTube,sum_item_parent_category_Apparel,sum_item_parent_category_Collections,sum_item_parent_category_Lifestyle,sum_item_parent_category_New,sum_item_parent_category_Other,sum_item_parent_category_Sale,sum_item_parent_category_ShopbyBrand,sum_item_parent_category_Stationery,sum_item_child_category_Bags,sum_item_child_category_CampusCollection,sum_item_child_category_Drinkware,sum_item_child_category_Google,sum_item_child_category_Kids,sum_item_child_category_MensUnisex,sum_item_child_category_Other,sum_item_child_category_SmallGoods,sum_item_child_category_Womens,sum_item_child_subcategory_Backpacks,sum_item_child_subcategory_ElectronicsAccessories,sum_item_child_subcategory_Infant,sum_item_child_subcategory_MensTShirts,sum_item_child_subcategory_MugsTumblers,sum_item_child_subcategory_Other,sum_item_child_subcategory_WaterBottles,max_event_params_engagement_time_msec,max_item_price_in_usd,max_item_promotions,max_item_quantity,max_item_refund_in_usd,max_item_refund_quantity,max_item_revenue_in_usd,avg_event_params_engagement_time_msec,avg_item_price_in_usd,avg_item_promotions,avg_item_quantity,avg_item_refund_in_usd,avg_item_refund_quantity,avg_item_revenue_in_usd,session_type


In [ ]:
df_t.head()

,user_pseudo_id,event_date,event_params_ga_session_number,ecommerce_transaction_id,sum_event_params_engagement_time_msec,sum_event_params_session_engaged,user_ltv_revenue,ecommerce_total_item_quantity,total_return_item_quantity,ecommerce_purchase_revenue_in_usd,ecommerce_refund_value_in_usd,ecommerce_tax_value_in_usd,ecommerce_unique_items,sum_item_price_in_usd,sum_item_promotions,sum_item_quantity,sum_item_refund_in_usd,sum_item_refund_quantity,sum_item_revenue_in_usd,sum_event_name_add_to_cart,sum_event_name_begin_checkout,sum_event_name_page_view,sum_event_name_scroll,sum_event_name_select_item,sum_event_name_user_engagement,sum_event_name_view_item,sum_device_category_desktop,sum_device_category_mobile,sum_device_mobile_brand_name_Apple,sum_device_mobile_brand_name_Google,sum_device_mobile_brand_name_Huawei,sum_device_mobile_brand_name_Microsoft,sum_device_mobile_brand_name_Mozilla,sum_device_mobile_brand_name_Samsung,sum_device_mobile_brand_name_Xiaomi,sum_device_mobile_model_name_Chrome,sum_device_mobile_model_name_ChromeBook,sum_device_mobile_model_name_Edge,sum_device_mobile_model_name_Firefox,sum_device_mobile_model_name_Safari,sum_device_mobile_model_name_iPad,sum_device_mobile_model_name_iPhone,sum_device_web_info_browser_AndroidWebview,sum_device_web_info_browser_Chrome,sum_device_web_info_browser_Edge,sum_device_web_info_browser_Firefox,sum_device_web_info_browser_Safari,sum_geo_country_Canada,sum_geo_country_France,sum_geo_country_India,sum_geo_country_Other,sum_geo_country_Spain,sum_geo_country_UnitedKingdom,sum_geo_country_UnitedStates,sum_traffic_source_medium_Other,sum_traffic_source_medium_cpc,sum_traffic_source_medium_organic,sum_traffic_source_medium_referral,sum_event_params_parent_page_Apparel,sum_event_params_parent_page_CampusCollection,sum_event_params_parent_page_CheckoutConfirmation,sum_event_params_parent_page_CheckoutYourInformation,sum_event_params_parent_page_EcoFriendly,sum_event_params_parent_page_Home,sum_event_params_parent_page_Lifestyle,sum_event_params_parent_page_New,sum_event_params_parent_page_Other,sum_event_params_parent_page_PaymentMethod,sum_event_params_parent_page_Sale,sum_event_params_parent_page_ShopbyBrand,sum_event_params_parent_page_ShoppingCart,sum_event_params_parent_page_Stationery,sum_event_params_child_page_Bags,sum_event_params_child_page_Drinkware,sum_event_params_child_page_Google,sum_event_params_child_page_Hats,sum_event_params_child_page_Kids,sum_event_params_child_page_MensUnisex,sum_event_params_child_page_Notebooks,sum_event_params_child_page_Other,sum_event_params_child_page_SmallGoods,sum_event_params_child_page_Socks,sum_event_params_child_page_Stickers,sum_event_params_child_page_Womens,sum_event_params_child_page_Writing,sum_event_params_child_page_YouTube,sum_item_parent_category_Apparel,sum_item_parent_category_Collections,sum_item_parent_category_Lifestyle,sum_item_parent_category_New,sum_item_parent_category_Other,sum_item_parent_category_Sale,sum_item_parent_category_ShopbyBrand,sum_item_parent_category_Stationery,sum_item_child_category_Bags,sum_item_child_category_CampusCollection,sum_item_child_category_Drinkware,sum_item_child_category_Google,sum_item_child_category_Kids,sum_item_child_category_MensUnisex,sum_item_child_category_Other,sum_item_child_category_SmallGoods,sum_item_child_category_Womens,sum_item_child_subcategory_Backpacks,sum_item_child_subcategory_ElectronicsAccessories,sum_item_child_subcategory_Infant,sum_item_child_subcategory_MensTShirts,sum_item_child_subcategory_MugsTumblers,sum_item_child_subcategory_Other,sum_item_child_subcategory_WaterBottles,max_event_params_engagement_time_msec,max_item_price_in_usd,max_item_promotions,max_item_quantity,max_item_refund_in_usd,max_item_refund_quantity,max_item_revenue_in_usd,avg_event_params_engagement_time_msec,avg_item_price_in_usd,avg_item_promotions,avg_item_quantity,avg_item_refund_in_usd,avg_item_refund_quantity,avg_item_revenue_in_usd,session_type
1,10092926.3786306416,2021-01-22,1

The transaction session table is now ready.

In [ ]:
############################################################
# CHECKPOINT - save table to BQ to save progress if needed #
############################################################

client = bigquery.Client(project = project_name)
table_id = 'return_prediction_ga4.step_3_transaction_sessions' # update your dataset id here as needed

job_config = bigquery.LoadJobConfig(
    write_disposition = bigquery.job.WriteDisposition.WRITE_TRUNCATE
)

job = client.load_table_from_dataframe(
    df_t, table_id, job_config = job_config
)

# Wait for the load job to complete.
job.result()

LoadJob<project=adl-analytics, location=US, id=d27fdcb6-96cd-4682-b022-b1a3e5bff8d5>

## Pre-Transaction Session Table

In [ ]:
# check col and row count
print(df_pt.shape)

(6271, 126)


In [ ]:
# count unique combos of user_pseudo_id, event_date and ecommerce_transaction_id
unique_combos_pt = df_pt.groupby(['user_pseudo_id', 'event_date','ecommerce_transaction_id']).size().reset_index(name='count').shape[0]
print(unique_combos_pt)

4686


In [ ]:
# let's also count unique combos of user_pseudo_id and ecommerce_transaction_id
unique_combos_pt = df_pt.groupby(['user_pseudo_id', 'ecommerce_transaction_id']).size().reset_index(name='count').shape[0]
print(unique_combos_pt)

2437


We wouldn't expect the above row count and unique combos to match yet since there are instances with multiple pre-transaction sessions for the same transaction. <br>

However, we do want to ultimately aggregate this table by user_pseudo_id and ecommerce_transaction_id so it can be left joined onto the transaction session table. <br>

We'll do that now by doing some aggregation as well as feature creation.

In [ ]:
df_pt.head()

,user_pseudo_id,event_date,event_params_ga_session_id,event_params_ga_session_number,ecommerce_transaction_id,sum_event_params_engagement_time_msec,sum_event_params_session_engaged,user_ltv_revenue,ecommerce_total_item_quantity,total_return_item_quantity,ecommerce_purchase_revenue_in_usd,ecommerce_refund_value_in_usd,ecommerce_tax_value_in_usd,ecommerce_unique_items,sum_item_price_in_usd,sum_item_promotions,sum_item_quantity,sum_item_refund_in_usd,sum_item_refund_quantity,sum_item_revenue_in_usd,sum_event_name_add_to_cart,sum_event_name_begin_checkout,sum_event_name_page_view,sum_event_name_scroll,sum_event_name_select_item,sum_event_name_user_engagement,sum_event_name_view_item,sum_device_category_desktop,sum_device_category_mobile,sum_device_mobile_brand_name_Apple,sum_device_mobile_brand_name_Google,sum_device_mobile_brand_name_Huawei,sum_device_mobile_brand_name_Microsoft,sum_device_mobile_brand_name_Mozilla,sum_device_mobile_brand_name_Samsung,sum_device_mobile_brand_name_Xiaomi,sum_device_mobile_model_name_Chrome,sum_device_mobile_model_name_ChromeBook,sum_device_mobile_model_name_Edge,sum_device_mobile_model_name_Firefox,sum_device_mobile_model_name_Safari,sum_device_mobile_model_name_iPad,sum_device_mobile_model_name_iPhone,sum_device_web_info_browser_AndroidWebview,sum_device_web_info_browser_Chrome,sum_device_web_info_browser_Edge,sum_device_web_info_browser_Firefox,sum_device_web_info_browser_Safari,sum_geo_country_Canada,sum_geo_country_France,sum_geo_country_India,sum_geo_country_Other,sum_geo_country_Spain,sum_geo_country_UnitedKingdom,sum_geo_country_UnitedStates,sum_traffic_source_medium_Other,sum_traffic_source_medium_cpc,sum_traffic_source_medium_organic,sum_traffic_source_medium_referral,sum_event_params_parent_page_Apparel,sum_event_params_parent_page_CampusCollection,sum_event_params_parent_page_CheckoutConfirmation,sum_event_params_parent_page_CheckoutYourInformation,sum_event_params_parent_page_EcoFriendly,sum_event_params_parent_page_Home,sum_event_params_parent_page_Lifestyle,sum_event_params_parent_page_New,sum_event_params_parent_page_Other,sum_event_params_parent_page_PaymentMethod,sum_event_params_parent_page_Sale,sum_event_params_parent_page_ShopbyBrand,sum_event_params_parent_page_ShoppingCart,sum_event_params_parent_page_Stationery,sum_event_params_child_page_Bags,sum_event_params_child_page_Drinkware,sum_event_params_child_page_Google,sum_event_params_child_page_Hats,sum_event_params_child_page_Kids,sum_event_params_child_page_MensUnisex,sum_event_params_child_page_Notebooks,sum_event_params_child_page_Other,sum_event_params_child_page_SmallGoods,sum_event_params_child_page_Socks,sum_event_params_child_page_Stickers,sum_event_params_child_page_Womens,sum_event_params_child_page_Writing,sum_event_params_child_page_YouTube,sum_item_parent_category_Apparel,sum_item_parent_category_Collections,sum_item_parent_category_Lifestyle,sum_item_parent_category_New,sum_item_parent_category_Other,sum_item_parent_category_Sale,sum_item_parent_category_ShopbyBrand,sum_item_parent_category_Stationery,sum_item_child_category_Bags,sum_item_child_category_CampusCollection,sum_item_child_category_Drinkware,sum_item_child_category_Google,sum_item_child_category_Kids,sum_item_child_category_MensUnisex,sum_item_child_category_Other,sum_item_child_category_SmallGoods,sum_item_child_category_Womens,sum_item_child_subcategory_Backpacks,sum_item_child_subcategory_ElectronicsAccessories,sum_item_child_subcategory_Infant,sum_item_child_subcategory_MensTShirts,sum_item_child_subcategory_MugsTumblers,sum_item_child_subcategory_Other,sum_item_child_subcategory_WaterBottles,max_event_params_engagement_time_msec,max_item_price_in_usd,max_item_promotions,max_item_quantity,max_item_refund_in_usd,max_item_refund_quantity,max_item_revenue_in_usd,avg_event_params_engagement_time_msec,avg_item_price_in_usd,avg_item_promotions,avg_item_quantity,avg_item_refund_in_usd,avg_item_refund_quantity,avg_item_revenue_in_usd,session_type
0,10092

Since no transaction occurred during the pre-transaction sessions, let's check that the transaction quantities and monetary values are null or 0 and drop those features from the pre-transaction sessions.

In [ ]:
df_pt.sum(numeric_only=True)

event_params_ga_session_number                              25645.000
sum_event_params_engagement_time_msec                  5639018682.000
sum_event_params_session_engaged                           477082.000
user_ltv_revenue                                                0.000
ecommerce_total_item_quantity                                   0.000
total_return_item_quantity                                      0.000
ecommerce_purchase_revenue_in_usd                               0.000
ecommerce_refund_value_in_usd                                   0.000
ecommerce_tax_value_in_usd                                      0.000
ecommerce_unique_items                                          0.000
sum_item_price_in_usd                                           0.000
sum_item_promotions                                             0.000
sum_item_quantity                                               0.000
sum_item_refund_in_usd                                          0.000
sum_item_refund_quan

As expected, the transaction-related values sum to zero, so we can safely drop these features from the pre-transaction sessions.

In [ ]:
df_pt = df_pt.drop([
    'user_ltv_revenue',
    'ecommerce_total_item_quantity',
    'total_return_item_quantity',
    'ecommerce_purchase_revenue_in_usd',
    'ecommerce_refund_value_in_usd',
    'ecommerce_tax_value_in_usd',
    'ecommerce_unique_items',
    'sum_item_price_in_usd',
    'sum_item_promotions',
    'sum_item_quantity',
    'sum_item_refund_in_usd',
    'sum_item_refund_quantity',
    'sum_item_revenue_in_usd',
    'max_item_price_in_usd',
    'max_item_promotions',
    'max_item_quantity',
    'max_item_refund_in_usd',
    'max_item_refund_quantity',
    'max_item_revenue_in_usd',
    'avg_item_price_in_usd',
    'avg_item_promotions',
    'avg_item_quantity',
    'avg_item_refund_in_usd',
    'avg_item_refund_quantity',
    'avg_item_revenue_in_usd'
], axis=1)

In [ ]:
df_pt_nunique = df_pt.groupby(['user_pseudo_id', 'ecommerce_transaction_id'])[
    'event_params_ga_session_id'
].agg('nunique').reset_index()

In [ ]:
# rename columns
df_pt_nunique.columns = [
    'user_pseudo_id',
    'ecommerce_transaction_id',
    'pre_nunique_event_params_ga_sessions'
]

In [ ]:
df_pt_nunique.shape

(2437, 3)

In [ ]:
# min value
df_pt_min = df_pt.groupby(['user_pseudo_id', 'ecommerce_transaction_id'])[
    'event_date'
].agg('min').reset_index()

In [ ]:
# rename columns
df_pt_min.columns = ['user_pseudo_id',
                      'ecommerce_transaction_id',
                      'pre_min_event_date']

In [ ]:
# standard deviation
df_pt_stdev = df_pt.groupby(['user_pseudo_id', 'ecommerce_transaction_id'])[
    'max_event_params_engagement_time_msec',
    'avg_event_params_engagement_time_msec',
].agg('std').reset_index()

In [ ]:
df_pt_stdev.columns = ['user_pseudo_id',
                      'ecommerce_transaction_id',
                      'pre_stdev_max_event_params_engagement_time_msec',
                      'pre_stdev_avg_event_params_engagement_time_msec']

In [ ]:
# max
df_pt_max = df_pt.groupby(['user_pseudo_id', 'ecommerce_transaction_id'])[
    'event_params_ga_session_number',
    'max_event_params_engagement_time_msec',
].agg('max').reset_index()

In [ ]:
df_pt_max.columns = ['user_pseudo_id',
                     'ecommerce_transaction_id',
                     'pre_max_event_params_ga_session_number',
                     'pre_max_event_params_engagement_time_msec']

In [ ]:
# mean
df_pt_average = df_pt.groupby(['user_pseudo_id', 'ecommerce_transaction_id'])[
    'avg_event_params_engagement_time_msec',
].agg('mean').reset_index()

In [ ]:
df_pt_average.columns = ['user_pseudo_id',
                         'ecommerce_transaction_id',
                         'pre_avg_event_params_engagement_time_msec']

In [ ]:
# sum
df_pt_sum = df_pt.groupby(['user_pseudo_id', 'ecommerce_transaction_id'])[
                  'sum_event_params_engagement_time_msec',
                  'sum_event_params_session_engaged',
                  'sum_event_name_add_to_cart',
                  'sum_event_name_begin_checkout',
                  'sum_event_name_page_view',
                  'sum_event_name_scroll',
                  'sum_event_name_select_item',
                  'sum_event_name_user_engagement',
                  'sum_event_name_view_item',
                  'sum_device_category_desktop',
                  'sum_device_category_mobile',
                  'sum_device_mobile_brand_name_Apple',
                  'sum_device_mobile_brand_name_Google',
                  'sum_device_mobile_brand_name_Huawei',
                  'sum_device_mobile_brand_name_Microsoft',
                  'sum_device_mobile_brand_name_Mozilla',
                  'sum_device_mobile_brand_name_Samsung',
                  'sum_device_mobile_brand_name_Xiaomi',
                  'sum_device_mobile_model_name_Chrome',
                  'sum_device_mobile_model_name_ChromeBook',
                  'sum_device_mobile_model_name_Edge',
                  'sum_device_mobile_model_name_Firefox',
                  'sum_device_mobile_model_name_Safari',
                  'sum_device_mobile_model_name_iPad',
                  'sum_device_mobile_model_name_iPhone',
                  'sum_device_web_info_browser_AndroidWebview',
                  'sum_device_web_info_browser_Chrome',
                  'sum_device_web_info_browser_Edge',
                  'sum_device_web_info_browser_Firefox',
                  'sum_device_web_info_browser_Safari',
                  'sum_geo_country_Canada',
                  'sum_geo_country_France',
                  'sum_geo_country_India',
                  'sum_geo_country_Other',
                  'sum_geo_country_Spain',
                  'sum_geo_country_UnitedKingdom',
                  'sum_geo_country_UnitedStates',
                  'sum_traffic_source_medium_Other',
                  'sum_traffic_source_medium_cpc',
                  'sum_traffic_source_medium_organic',
                  'sum_traffic_source_medium_referral',
                  'sum_event_params_parent_page_Apparel',
                  'sum_event_params_parent_page_CampusCollection',
                  'sum_event_params_parent_page_CheckoutConfirmation',
                  'sum_event_params_parent_page_CheckoutYourInformation',
                  'sum_event_params_parent_page_EcoFriendly',
                  'sum_event_params_parent_page_Home',
                  'sum_event_params_parent_page_Lifestyle',
                  'sum_event_params_parent_page_New',
                  'sum_event_params_parent_page_Other',
                  'sum_event_params_parent_page_PaymentMethod',
                  'sum_event_params_parent_page_Sale',
                  'sum_event_params_parent_page_ShopbyBrand',
                  'sum_event_params_parent_page_ShoppingCart',
                  'sum_event_params_parent_page_Stationery',
                  'sum_event_params_child_page_Bags',
                  'sum_event_params_child_page_Drinkware',
                  'sum_event_params_child_page_Google',
                  'sum_event_params_child_page_Hats',
                  'sum_event_params_child_page_Kids',
                  'sum_event_params_child_page_MensUnisex',
                  'sum_event_params_child_page_Notebooks',
                  'sum_event_params_child_page_Other',
                  'sum_event_params_child_page_SmallGoods',
                  'sum_event_params_child_page_Socks',
                  'sum_event_params_child_page_Stickers',
                  'sum_event_params_child_page_Womens',
                  'sum_event_params_child_page_Writing',
                  'sum_event_params_child_page_YouTube',
                  'sum_item_parent_category_Apparel',
                  'sum_item_parent_category_Collections',
                  'sum_item_parent_category_Lifestyle',
                  'sum_item_parent_category_New',
                  'sum_item_parent_category_Other',
                  'sum_item_parent_category_Sale',
                  'sum_item_parent_category_ShopbyBrand',
                  'sum_item_parent_category_Stationery',
                  'sum_item_child_category_Bags',
                  'sum_item_child_category_CampusCollection',
                  'sum_item_child_category_Drinkware',
                  'sum_item_child_category_Google',
                  'sum_item_child_category_Kids',
                  'sum_item_child_category_MensUnisex',
                  'sum_item_child_category_Other',
                  'sum_item_child_category_SmallGoods',
                  'sum_item_child_category_Womens',
                  'sum_item_child_subcategory_Backpacks',
                  'sum_item_child_subcategory_ElectronicsAccessories',
                  'sum_item_child_subcategory_Infant',
                  'sum_item_child_subcategory_MensTShirts',
                  'sum_item_child_subcategory_MugsTumblers',
                  'sum_item_child_subcategory_Other',
                  'sum_item_child_subcategory_WaterBottles'
].agg('sum').reset_index()

In [ ]:
df_pt_sum.columns = ['user_pseudo_id',
                     'ecommerce_transaction_id',
                     'pre_sum_event_params_engagement_time_msec',
                     'pre_sum_event_params_session_engaged',
                     'pre_sum_event_name_add_to_cart',
                     'pre_sum_event_name_begin_checkout',
                     'pre_sum_event_name_page_view',
                     'pre_sum_event_name_scroll',
                     'pre_sum_event_name_select_item',
                     'pre_sum_event_name_user_engagement',
                     'pre_sum_event_name_view_item',
                     'pre_sum_device_category_desktop',
                     'pre_sum_device_category_mobile',
                     'pre_sum_device_mobile_brand_name_Apple',
                     'pre_sum_device_mobile_brand_name_Google',
                     'pre_sum_device_mobile_brand_name_Huawei',
                     'pre_sum_device_mobile_brand_name_Microsoft',
                     'pre_sum_device_mobile_brand_name_Mozilla',
                     'pre_sum_device_mobile_brand_name_Samsung',
                     'pre_sum_device_mobile_brand_name_Xiaomi',
                     'pre_sum_device_mobile_model_name_Chrome',
                     'pre_sum_device_mobile_model_name_ChromeBook',
                     'pre_sum_device_mobile_model_name_Edge',
                     'pre_sum_device_mobile_model_name_Firefox',
                     'pre_sum_device_mobile_model_name_Safari',
                     'pre_sum_device_mobile_model_name_iPad',
                     'pre_sum_device_mobile_model_name_iPhone',
                     'pre_sum_device_web_info_browser_AndroidWebview',
                     'pre_sum_device_web_info_browser_Chrome',
                     'pre_sum_device_web_info_browser_Edge',
                     'pre_sum_device_web_info_browser_Firefox',
                     'pre_sum_device_web_info_browser_Safari',
                     'pre_sum_geo_country_Canada',
                     'pre_sum_geo_country_France',
                     'pre_sum_geo_country_India',
                     'pre_sum_geo_country_Other',
                     'pre_sum_geo_country_Spain',
                     'pre_sum_geo_country_UnitedKingdom',
                     'pre_sum_geo_country_UnitedStates',
                     'pre_sum_traffic_source_medium_Other',
                     'pre_sum_traffic_source_medium_cpc',
                     'pre_sum_traffic_source_medium_organic',
                     'pre_sum_traffic_source_medium_referral',
                     'pre_sum_event_params_parent_page_Apparel',
                     'pre_sum_event_params_parent_page_CampusCollection',
                     'pre_sum_event_params_parent_page_CheckoutConfirmation',
                     'pre_sum_event_params_parent_page_CheckoutYourInformation',
                     'pre_sum_event_params_parent_page_EcoFriendly',
                     'pre_sum_event_params_parent_page_Home',
                     'pre_sum_event_params_parent_page_Lifestyle',
                     'pre_sum_event_params_parent_page_New',
                     'pre_sum_event_params_parent_page_Other',
                     'pre_sum_event_params_parent_page_PaymentMethod',
                     'pre_sum_event_params_parent_page_Sale',
                     'pre_sum_event_params_parent_page_ShopbyBrand',
                     'pre_sum_event_params_parent_page_ShoppingCart',
                     'pre_sum_event_params_parent_page_Stationery',
                     'pre_sum_event_params_child_page_Bags',
                     'pre_sum_event_params_child_page_Drinkware',
                     'pre_sum_event_params_child_page_Google',
                     'pre_sum_event_params_child_page_Hats',
                     'pre_sum_event_params_child_page_Kids',
                     'pre_sum_event_params_child_page_MensUnisex',
                     'pre_sum_event_params_child_page_Notebooks',
                     'pre_sum_event_params_child_page_Other',
                     'pre_sum_event_params_child_page_SmallGoods',
                     'pre_sum_event_params_child_page_Socks',
                     'pre_sum_event_params_child_page_Stickers',
                     'pre_sum_event_params_child_page_Womens',
                     'pre_sum_event_params_child_page_Writing',
                     'pre_sum_event_params_child_page_YouTube',
                     'pre_sum_item_parent_category_Apparel',
                     'pre_sum_item_parent_category_Collections',
                     'pre_sum_item_parent_category_Lifestyle',
                     'pre_sum_item_parent_category_New',
                     'pre_sum_item_parent_category_Other',
                     'pre_sum_item_parent_category_Sale',
                     'pre_sum_item_parent_category_ShopbyBrand',
                     'pre_sum_item_parent_category_Stationery',
                     'pre_sum_item_child_category_Bags',
                     'pre_sum_item_child_category_CampusCollection',
                     'pre_sum_item_child_category_Drinkware',
                     'pre_sum_item_child_category_Google',
                     'pre_sum_item_child_category_Kids',
                     'pre_sum_item_child_category_MensUnisex',
                     'pre_sum_item_child_category_Other',
                     'pre_sum_item_child_category_SmallGoods',
                     'pre_sum_item_child_category_Womens',
                     'pre_sum_item_child_subcategory_Backpacks',
                     'pre_sum_item_child_subcategory_ElectronicsAccessories',
                     'pre_sum_item_child_subcategory_Infant',
                     'pre_sum_item_child_subcategory_MensTShirts',
                     'pre_sum_item_child_subcategory_MugsTumblers',
                     'pre_sum_item_child_subcategory_Other',
                     'pre_sum_item_child_subcategory_WaterBottles']

In [ ]:
# combine all pre-transaction session dfs
dfs = [df_pt_nunique, df_pt_min, df_pt_stdev, df_pt_max, df_pt_average, df_pt_sum]

# Define the columns on which to merge
merge_cols = ['user_pseudo_id','ecommerce_transaction_id']

# Use reduce to merge the DataFrames one by one
df_pt_merged = reduce(lambda left, right: pd.merge(left, right, on=merge_cols), dfs)

In [ ]:
df_pt_merged.shape

(2437, 102)

In [ ]:
############################################################
# CHECKPOINT - save table to BQ to save progress if needed #
############################################################

client = bigquery.Client(project = project_name)
table_id = 'return_prediction_ga4.step_3_pre_transaction_sessions' # update your dataset id here as needed

job_config = bigquery.LoadJobConfig(
    write_disposition = bigquery.job.WriteDisposition.WRITE_TRUNCATE
)

job = client.load_table_from_dataframe(
    df_pt_merged, table_id, job_config = job_config
)

# Wait for the load job to complete.
job.result()

LoadJob<project=adl-analytics, location=US, id=11c1af1d-6fcd-4383-ad8e-424d62889054>

In [ ]:
df_t.shape

(4466, 125)

In [ ]:
df_pt_merged.shape

(2437, 102)

In [ ]:
# next, merge df_t and df_pt_merged
df_all = pd.merge(
    df_t,
    df_pt_merged,
    how = 'left',
    on = ['user_pseudo_id','ecommerce_transaction_id']
    )

In [ ]:
df_all.shape

(4466, 225)

In [ ]:
###############################################
######## CHECKPOINT - save table to BQ ########
###############################################

client = bigquery.Client(project = project_name)
table_id = 'return_prediction_ga4.step_3_all_sessions' # update your dataset id here as needed

job_config = bigquery.LoadJobConfig(
    write_disposition = bigquery.job.WriteDisposition.WRITE_TRUNCATE
)

job = client.load_table_from_dataframe(
    df_all, table_id, job_config = job_config
)

# Wait for the load job to complete.
job.result()

LoadJob<project=adl-analytics, location=US, id=f333fb6e-a2ad-4c42-bcb7-a33cbf672ec0>

In [ ]:
# move event date closer to front of df for easier spot checking
cols = list(df_all.columns)
cols.remove('event_date')
cols.remove('user_pseudo_id')
cols = ['user_pseudo_id', 'event_date'] + cols
df_all = df_all[cols]

In [ ]:
df_all.head()

,user_pseudo_id,event_date,event_params_ga_session_number,ecommerce_transaction_id,sum_event_params_engagement_time_msec,sum_event_params_session_engaged,user_ltv_revenue,ecommerce_total_item_quantity,total_return_item_quantity,ecommerce_purchase_revenue_in_usd,ecommerce_refund_value_in_usd,ecommerce_tax_value_in_usd,ecommerce_unique_items,sum_item_price_in_usd,sum_item_promotions,sum_item_quantity,sum_item_refund_in_usd,sum_item_refund_quantity,sum_item_revenue_in_usd,sum_event_name_add_to_cart,sum_event_name_begin_checkout,sum_event_name_page_view,sum_event_name_scroll,sum_event_name_select_item,sum_event_name_user_engagement,sum_event_name_view_item,sum_device_category_desktop,sum_device_category_mobile,sum_device_mobile_brand_name_Apple,sum_device_mobile_brand_name_Google,sum_device_mobile_brand_name_Huawei,sum_device_mobile_brand_name_Microsoft,sum_device_mobile_brand_name_Mozilla,sum_device_mobile_brand_name_Samsung,sum_device_mobile_brand_name_Xiaomi,sum_device_mobile_model_name_Chrome,sum_device_mobile_model_name_ChromeBook,sum_device_mobile_model_name_Edge,sum_device_mobile_model_name_Firefox,sum_device_mobile_model_name_Safari,sum_device_mobile_model_name_iPad,sum_device_mobile_model_name_iPhone,sum_device_web_info_browser_AndroidWebview,sum_device_web_info_browser_Chrome,sum_device_web_info_browser_Edge,sum_device_web_info_browser_Firefox,sum_device_web_info_browser_Safari,sum_geo_country_Canada,sum_geo_country_France,sum_geo_country_India,sum_geo_country_Other,sum_geo_country_Spain,sum_geo_country_UnitedKingdom,sum_geo_country_UnitedStates,sum_traffic_source_medium_Other,sum_traffic_source_medium_cpc,sum_traffic_source_medium_organic,sum_traffic_source_medium_referral,sum_event_params_parent_page_Apparel,sum_event_params_parent_page_CampusCollection,sum_event_params_parent_page_CheckoutConfirmation,sum_event_params_parent_page_CheckoutYourInformation,sum_event_params_parent_page_EcoFriendly,sum_event_params_parent_page_Home,sum_event_params_parent_page_Lifestyle,sum_event_params_parent_page_New,sum_event_params_parent_page_Other,sum_event_params_parent_page_PaymentMethod,sum_event_params_parent_page_Sale,sum_event_params_parent_page_ShopbyBrand,sum_event_params_parent_page_ShoppingCart,sum_event_params_parent_page_Stationery,sum_event_params_child_page_Bags,sum_event_params_child_page_Drinkware,sum_event_params_child_page_Google,sum_event_params_child_page_Hats,sum_event_params_child_page_Kids,sum_event_params_child_page_MensUnisex,sum_event_params_child_page_Notebooks,sum_event_params_child_page_Other,sum_event_params_child_page_SmallGoods,sum_event_params_child_page_Socks,sum_event_params_child_page_Stickers,sum_event_params_child_page_Womens,sum_event_params_child_page_Writing,sum_event_params_child_page_YouTube,sum_item_parent_category_Apparel,sum_item_parent_category_Collections,sum_item_parent_category_Lifestyle,sum_item_parent_category_New,sum_item_parent_category_Other,sum_item_parent_category_Sale,sum_item_parent_category_ShopbyBrand,sum_item_parent_category_Stationery,sum_item_child_category_Bags,sum_item_child_category_CampusCollection,sum_item_child_category_Drinkware,sum_item_child_category_Google,sum_item_child_category_Kids,sum_item_child_category_MensUnisex,sum_item_child_category_Other,sum_item_child_category_SmallGoods,sum_item_child_category_Womens,sum_item_child_subcategory_Backpacks,sum_item_child_subcategory_ElectronicsAccessories,sum_item_child_subcategory_Infant,sum_item_child_subcategory_MensTShirts,sum_item_child_subcategory_MugsTumblers,sum_item_child_subcategory_Other,sum_item_child_subcategory_WaterBottles,max_event_params_engagement_time_msec,max_item_price_in_usd,max_item_promotions,max_item_quantity,max_item_refund_in_usd,max_item_refund_quantity,max_item_revenue_in_usd,avg_event_params_engagement_time_msec,avg_item_price_in_usd,avg_item_promotions,avg_item_quantity,avg_item_refund_in_usd,avg_item_refund_quantity,avg_item_revenue_in_usd,session_type,pre_nunique_event_params_ga_sessio

In [ ]:
df_all = df_all.rename(columns={
    'event_date': 'transaction_date',
    'event_params_ga_session_number':'transaction_ga_session_number'
    })

In [ ]:
# create feature for days between first session and transaction
df_all['days_first_session_to_transaction'] = (pd.to_datetime(df_all['transaction_date']) - pd.to_datetime(df_all['pre_min_event_date'])).dt.days
# drop pre_min_event_date column
df_all.drop('pre_min_event_date', axis=1, inplace=True)

In [ ]:
# check missing values
df_all.isna().sum()

user_pseudo_id                                                 0
transaction_date                                               0
transaction_ga_session_number                                  0
ecommerce_transaction_id                                       0
sum_event_params_engagement_time_msec                          0
sum_event_params_session_engaged                               0
user_ltv_revenue                                               0
ecommerce_total_item_quantity                                  0
total_return_item_quantity                                     0
ecommerce_purchase_revenue_in_usd                              0
ecommerce_refund_value_in_usd                                  0
ecommerce_tax_value_in_usd                                     0
ecommerce_unique_items                                         0
sum_item_price_in_usd                                          0
sum_item_promotions                                            0
sum_item_quantity        

In [ ]:
# replace missing values with 0
df_all = df_all.fillna(0)

In [ ]:
############################################################
# CHECKPOINT - save table to BQ to save progress if needed #
############################################################

client = bigquery.Client(project = project_name)
table_id = 'return_prediction_ga4.step_3_final' # update your dataset id here as needed

job_config = bigquery.LoadJobConfig(
    write_disposition = bigquery.job.WriteDisposition.WRITE_TRUNCATE
)

job = client.load_table_from_dataframe(
    df_all, table_id, job_config = job_config
)

# Wait for the load job to complete.
job.result()

LoadJob<project=adl-analytics, location=US, id=92438f7c-8e02-4226-9738-e7ba50a5bbac>